In [1]:
from  google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import os
import glob
import locale
#locale.setlocale(locale.LC_ALL, 'es_ES')
import calendar
import openpyxl as xl

In [3]:
file_path = "/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/COB 2018-2022_EESS_CORREJIDOS.xlsx"

# Funciones

In [4]:
def define_multilevel_col(df, enfermedad, rango_de_edad):
  month_cols = pd.MultiIndex.from_tuples([
                                    (f"{rango_de_edad}",f"{enfermedad}","ENERO", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","ENERO", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","FEBRERO", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","FEBRERO", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","MARZO", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","MARZO", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","ABRIL", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","ABRIL", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","MAYO", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","MAYO", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","JUNIO", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","JUNIO", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","JULIO", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","JULIO", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","AGOSTO", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","AGOSTO", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","SETIEMBRE", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","SETIEMBRE", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","OCTUBRE", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","OCTUBRE", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","NOVIEMBRE", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","NOVIEMBRE", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","DICIEMBRE", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","DICIEMBRE", "%")])
  return pd.DataFrame(df.values, columns=month_cols)

In [5]:
def preprocessing_enf_edad(df, col, enfermedad, rango_de_edad):
  df_final = df.iloc[:, col:col+24]
  df_final.columns = df_final.iloc[0]
  return define_multilevel_col(df_final[1:], enfermedad, rango_de_edad)

In [6]:
def preprocessing_total(file_path, sheet_name, departamento):
  #leer archivos
  df_cobertura = pd.read_excel(file_path, sheet_name = sheet_name)

  #Preprocesamiento de centro de salud
  df_centros_de_salud = df_cobertura[df_cobertura['Marca'] != 'x'][['ESTABLECIMIENTO DE SALUD']]
  df_centros_de_salud.dropna(inplace = True)
  #df_centros_de_salud['ESTABLECIMIENTO DE SALUD'] = df_centros_de_salud['ESTABLECIMIENTO DE SALUD'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
  #df_centros_de_salud['ESTABLECIMIENTO DE SALUD'] = df_centros_de_salud['ESTABLECIMIENTO DE SALUD'].str.strip()
  #df_centros_de_salud['ESTABLECIMIENTO DE SALUD'] = df_centros_de_salud['ESTABLECIMIENTO DE SALUD'].str.replace('    ',' ')
  #df_centros_de_salud['ESTABLECIMIENTO DE SALUD'] = df_centros_de_salud['ESTABLECIMIENTO DE SALUD'].str.replace('  ',' ')
  df_centros_de_salud.rename(columns={'ESTABLECIMIENTO DE SALUD': 'ESTABLECIMIENTO_DE_SALUD'}, inplace=True)

  # agregando año
  df_centros_de_salud['AÑO'] = int(sheet_name[:4])

  #Preprocesamiento enfermedades
  #BCG-RN
  enfermedad = 'BCG'
  rango_de_edad = 'RN'
  df_rn_BCG_final = preprocessing_enf_edad(df_cobertura, 3, enfermedad, rango_de_edad)
  # HVB-RN
  enfermedad = 'HVB'
  rango_de_edad = 'RN'
  df_rn_HVB_final = preprocessing_enf_edad(df_cobertura, 29, enfermedad, rango_de_edad)
  # APO-<1AÑO
  enfermedad = 'APO'
  rango_de_edad = '<1_AÑO'
  df_m1_apo_final = preprocessing_enf_edad(df_cobertura, 56, enfermedad, rango_de_edad)
  # PENTA-<1AÑO
  enfermedad = 'PENTA'
  rango_de_edad = '<1_AÑO'
  df_m1_penta_final = preprocessing_enf_edad(df_cobertura, 82, enfermedad, rango_de_edad)
  # ROTA-<1AÑO
  enfermedad = 'ROTA'
  rango_de_edad = '<1_AÑO'
  df_m1_rota_final = preprocessing_enf_edad(df_cobertura, 108, enfermedad, rango_de_edad)
  # NEUMO-1AÑO
  enfermedad = 'NEUMO'
  rango_de_edad = '1_AÑO'
  df_m1_neumo_final = preprocessing_enf_edad(df_cobertura, 135, enfermedad, rango_de_edad)
  # SPR-1AÑO
  enfermedad = 'SPR'
  rango_de_edad = '1_AÑO'
  df_m1_spr_final = preprocessing_enf_edad(df_cobertura, 161, enfermedad, rango_de_edad)
  # APT-1AÑO (en 2021 DPT?)
  enfermedad = 'APT'
  rango_de_edad = '1_AÑO'
  df_m1_dpt_final = preprocessing_enf_edad(df_cobertura, 187, enfermedad, rango_de_edad)
  # APO_REF-1AÑO
  enfermedad = 'APO_REF'
  rango_de_edad = '1_AÑO'
  df_m1_aporef_final = preprocessing_enf_edad(df_cobertura, 213, enfermedad, rango_de_edad)
  #AMA-1AÑO
  enfermedad = 'AMA'
  rango_de_edad = '1_AÑO'
  df_m1_ama_final = preprocessing_enf_edad(df_cobertura, 239, enfermedad, rango_de_edad)
  #VARI-1AÑO
  enfermedad = 'VARI'
  rango_de_edad = '1_AÑO'
  df_m1_vari_final = preprocessing_enf_edad(df_cobertura, 265, enfermedad, rango_de_edad)
  #INFLUENZA-2AÑOS
  enfermedad = 'INFLUENZA'
  rango_de_edad = '2_AÑOS'
  df_2_infllu_final = preprocessing_enf_edad(df_cobertura, 292, enfermedad, rango_de_edad)
  #INFLUENZA-3AÑOS
  enfermedad = 'INFLUENZA'
  rango_de_edad = '3_AÑOS'
  df_3_infllu_final = preprocessing_enf_edad(df_cobertura, 319, enfermedad, rango_de_edad)
  #DPT2-4AÑOS
  enfermedad = 'DPT2'
  rango_de_edad = '4_AÑOS'
  df_4_DPT2_final = preprocessing_enf_edad(df_cobertura, 346, enfermedad, rango_de_edad)
  #APO2-4AÑOS
  enfermedad = 'APO2'
  rango_de_edad = '4_AÑOS'
  df_4_APO2_final = preprocessing_enf_edad(df_cobertura, 372, enfermedad, rango_de_edad)

  #Merge
  df_ano_enf = pd.concat([df_rn_BCG_final, df_rn_HVB_final, df_m1_apo_final, df_m1_penta_final, df_m1_rota_final, df_m1_neumo_final, df_m1_spr_final, df_m1_dpt_final, df_m1_aporef_final, df_m1_ama_final, df_m1_vari_final, df_2_infllu_final, df_3_infllu_final, df_4_DPT2_final, df_4_APO2_final] , axis = 1)
  df_ano_enf.index = df_ano_enf.index +1
  df_final_1 = pd.concat([df_centros_de_salud,df_ano_enf],axis = 1)
  df_final_1['DEPARTAMENTO'] = departamento
  df_final_1.dropna(inplace = True)
  return df_final_1


In [7]:
def to_multindex(df):
  tuple_columns = [col for col in df.columns if isinstance(col, tuple)]
  non_tuple_columns = [col for col in df.columns if not isinstance(col, tuple)]
  #import pdb; pdb.set_trace()
  if tuple_columns:
    multi_index = pd.MultiIndex.from_tuples(tuple_columns)
    df.columns = pd.Index(non_tuple_columns + [multi_index])
  return df

In [8]:
def cob_formatting (df):
  df.set_index(["ESTABLECIMIENTO_DE_SALUD", "AÑO", "DEPARTAMENTO"], drop = True, inplace = True, append = True)
  df.columns = pd.MultiIndex.from_tuples(df.columns)
  df.index.set_names(["INDEX", "CENTRO_SALUD", "AÑO","DEPARTAMENTO"], inplace = True)
  df.columns.names = ["RANGO_EDAD","EFERMERDAD","MES","COLUMNS"]
  df_test_onel = df.stack(level = [0,1,2])
  df_test_onel.reset_index(inplace = True)
  return df_test_onel.drop("INDEX",axis = 1)

# EJECUCION

In [9]:
df_test_2022 = preprocessing_total(file_path, "2022XMES","UCAYALI")
df_test_2021 = preprocessing_total(file_path, "2021XMES","UCAYALI")
df_test_2020 = preprocessing_total(file_path, "2020XMES","UCAYALI")
df_test_2019 = preprocessing_total(file_path, "2019XMES","UCAYALI")

In [10]:
df_test_onel_2019 = cob_formatting(df_test_2019)
df_test_onel_2020 = cob_formatting(df_test_2020)
df_test_onel_2021 = cob_formatting(df_test_2021)
df_test_onel_2022 = cob_formatting(df_test_2022)

In [34]:
df_test_onel_all_years = pd.concat([df_test_onel_2019,df_test_onel_2020,df_test_onel_2021,df_test_onel_2022])
df_test_onel_all_years["AÑO"] = df_test_onel_all_years["AÑO"].astype("int64")
df_test_onel_all_years["CANTIDAD"] = df_test_onel_all_years["CANTIDAD"].astype("int64")
df_test_onel_all_years.rename(columns = {"AÑO":"ANIO", "EFERMERDAD":"VACUNA"}, inplace = True)
df_test_onel_all_years['RANGO_EDAD'] = df_test_onel_all_years['RANGO_EDAD'].str.replace('AÑO', 'ANIO')
df_test_onel_all_years

COLUMNS,CENTRO_SALUD,ANIO,DEPARTAMENTO,RANGO_EDAD,VACUNA,MES,%,CANTIDAD
0,SANIDAD AEREA,2019,UCAYALI,1_ANIO,AMA,ABRIL,0,0
1,SANIDAD AEREA,2019,UCAYALI,1_ANIO,AMA,AGOSTO,0,0
2,SANIDAD AEREA,2019,UCAYALI,1_ANIO,AMA,DICIEMBRE,0,0
3,SANIDAD AEREA,2019,UCAYALI,1_ANIO,AMA,ENERO,0,0
4,SANIDAD AEREA,2019,UCAYALI,1_ANIO,AMA,FEBRERO,0,0
...,...,...,...,...,...,...,...,...
38695,HOSPITAL AMAZONICO YARINACOCHA,2022,UCAYALI,RN,HVB,MARZO,0,289
38696,HOSPITAL AMAZONICO YARINACOCHA,2022,UCAYALI,RN,HVB,MAYO,0,292
38697,HOSPITAL AMAZONICO YARINACOCHA,2022,UCAYALI,RN,HVB,NOVIEMBRE,0,300
38698,HOSPITAL AMAZONICO YARINACOCHA,2022,UCAYALI,RN,HVB,OCTUBRE,0,274


In [35]:
all_years_personal_salud = pd.read_csv("/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/Coberturas_Ucayali/Complete/consolidado_allYears_UCAYALI_final_2.csv")
all_years_personal_salud

<ipython-input-35-f11e9078f2fd>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_years_personal_salud = pd.read_csv("/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/Coberturas_Ucayali/Complete/consolidado_allYears_UCAYALI_final_2.csv")


,CENTRO_SALUD,ANIO,DEPARTAMENTO,RANGO_EDAD,VACUNA,MES,COBERTURA,CANTIDAD,PROVINCIA,DISTRITO,RED,MICRORRED,CANTIDAD DE PERSONAL,POBLACION,PRESUPUESTO
0,SANIDAD AEREA,2019,UCAYALI,1_ANIO,AMA,ABRIL,0,0.0,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,NaN,115625,998497
1,SANIDAD AEREA,2019,UCAYALI,1_ANIO,AMA,AGOSTO,0,0.0,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,2.0,115625,3650033
2,SANIDAD AEREA,2019,UCAYALI,1_ANIO,AMA,DICIEMBRE,0,0.0,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,2.0,115625,1953340
3,SANIDAD AEREA,2019,UCAYALI,1_ANIO,AMA,ENERO,0,0.0,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,1.0,115625,924269
4,SANIDAD AEREA,2019,UCAYALI,1_ANIO,AMA,FEBRERO,0,0.0,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,1.0,115625,1000320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127255,ZORRILLOS,2022,UCAYALI,RN,HVB,MARZO,0.0,0.0,PADRE ABAD,CURIMANA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,1.0,10946,784465
127256,ZORRILLOS,2022,UCAYALI,RN,HVB,MAYO,0.0,0.0,PADRE ABAD,CURIMANA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,1.0,10946,760212
127257,ZORRILLOS,2022,UCAYALI,RN,HVB,NOVIEMBRE,0.0,0.0,PADRE ABAD,CURIMANA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,NaN,10946,677800
127258,ZORRILLOS,2022,UCAYALI,RN,HVB,OCTUBRE,0.0,0.0,PADRE ABAD,CURIMANA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,NaN,10946,541898


In [39]:
df_test = all_years_personal_salud.merge(df_test_onel_all_years, how = "left", on =["CENTRO_SALUD","ANIO","RANGO_EDAD","VACUNA","MES"])
df_test["COBERTURA"] = df_test["%"]
df_test.drop(labels = ["CANTIDAD_x", "DEPARTAMENTO_y"], axis = 1, inplace = True)
df_test.rename(columns={"CANTIDAD_y":"CANTIDAD","DEPARTAMENTO_x":"DEPARTAMENTO"},inplace = True)
df_test

,CENTRO_SALUD,ANIO,DEPARTAMENTO,RANGO_EDAD,VACUNA,MES,COBERTURA,PROVINCIA,DISTRITO,RED,MICRORRED,CANTIDAD DE PERSONAL,POBLACION,PRESUPUESTO,%,CANTIDAD
0,SANIDAD AEREA,2019,UCAYALI,1_ANIO,AMA,ABRIL,0,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,NaN,115625,998497,0,0
1,SANIDAD AEREA,2019,UCAYALI,1_ANIO,AMA,AGOSTO,0,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,2.0,115625,3650033,0,0
2,SANIDAD AEREA,2019,UCAYALI,1_ANIO,AMA,DICIEMBRE,0,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,2.0,115625,1953340,0,0
3,SANIDAD AEREA,2019,UCAYALI,1_ANIO,AMA,ENERO,0,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,1.0,115625,924269,0,0
4,SANIDAD AEREA,2019,UCAYALI,1_ANIO,AMA,FEBRERO,0,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,1.0,115625,1000320,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127255,ZORRILLOS,2022,UCAYALI,RN,HVB,MARZO,0,PADRE ABAD,CURIMANA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,1.0,10946,784465,0,0
127256,ZORRILLOS,2022,UCAYALI,RN,HVB,MAYO,0,PADRE ABAD,CURIMANA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,1.0,10946,760212,0,0
127257,ZORRILLOS,2022,UCAYALI,RN,HVB,NOVIEMBRE,0,PADRE ABAD,CURIMANA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,NaN,10946,677800,0,0
127258,ZORRILLOS,2022,UCAYALI,RN,HVB,OCTUBRE,0,PADRE ABAD,CURIMANA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,NaN,10946,541898,0,0


In [40]:
df_test.to_csv("/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/Coberturas_Ucayali/Complete/consolidado_allYears_UCAYALI_final_3.csv", index = False)

In [37]:
df_test[df_test["CANTIDAD_y"].isna()]

,CENTRO_SALUD,ANIO,DEPARTAMENTO_x,RANGO_EDAD,VACUNA,MES,COBERTURA,CANTIDAD_x,PROVINCIA,DISTRITO,RED,MICRORRED,CANTIDAD DE PERSONAL,POBLACION,PRESUPUESTO,DEPARTAMENTO_y,%,CANTIDAD_y


In [ ]:
df_test_onel_2019.to_csv("/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/cob_formateado_2019.csv", index = False, encoding = "utf-8")
df_test_onel_2020.to_csv("/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/cob_formateado_2020.csv", index = False, encoding = "utf-8")
df_test_onel_2021.to_csv("/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/cob_formateado_2021.csv", index = False, encoding = "utf-8")
df_test_onel_2022.to_csv("/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/cob_formateado_2022.csv", index = False, encoding = "utf-8")

In [ ]:
#df_test_2020_onel.reset_index(["INDEX","AÑO",	"DEPARTAMENTO",	"RANGO_EDAD",	"EFERMERDAD"], inplace= True)
#df_test_2020_onel.reset_index(["INDEX"], inplace= True)
df_test_2020_onel.reset_index(inplace = True)

In [ ]:
df_test_2020_onel

COLUMNS,INDEX,CENTRO_SALUD,AÑO,DEPARTAMENTO,RANGO_EDAD,EFERMERDAD,MES,%,CANTIDAD
0,3,CENTRO AMERICA,2020.0,UCAYALI,1_AÑO,AMA,ABRIL,0,0
1,3,CENTRO AMERICA,2020.0,UCAYALI,1_AÑO,AMA,AGOSTO,4.292929,17
2,3,CENTRO AMERICA,2020.0,UCAYALI,1_AÑO,AMA,DICIEMBRE,1.010101,4
3,3,CENTRO AMERICA,2020.0,UCAYALI,1_AÑO,AMA,ENERO,2.525253,10
4,3,CENTRO AMERICA,2020.0,UCAYALI,1_AÑO,AMA,FEBRERO,3.282828,13
...,...,...,...,...,...,...,...,...,...
37615,231,HOSPITAL AMAZONICO YARINACOCH,2020.0,UCAYALI,RN,HVB,MARZO,0,0
37616,231,HOSPITAL AMAZONICO YARINACOCH,2020.0,UCAYALI,RN,HVB,MAYO,0,0
37617,231,HOSPITAL AMAZONICO YARINACOCH,2020.0,UCAYALI,RN,HVB,NOVIEMBRE,0,0
37618,231,HOSPITAL AMAZONICO YARINACOCH,2020.0,UCAYALI,RN,HVB,OCTUBRE,0,0


In [ ]:
df_revision = df_summary_2020.groupby(by =['AÑO', 'MES','DESCRIPCION_ESTABLECIMIENTO']).agg(pd.Series.count)
df_revision[df_revision["COUNT"]>1]

,,,DEPARTAMENTO,PROVINCIA,DISTRITO,RED,MICRORRED,COUNT
AÑO,MES,DESCRIPCION_ESTABLECIMIENTO,,,,,,


In [ ]:
df_summary_2020[df_summary_2020["DESCRIPCION_ESTABLECIMIENTO"] == "NUEVO PARAISO"]

In [ ]:
df_total_2020_f = df_test_2020_onel.merge(df_summary, left_on=["AÑO","MES","CENTRO_SALUD"], right_on=["AÑO","MES", "DESCRIPCION_ESTABLECIMIENTO"], how = "left")
df_total_2020_f.rename(columns = {"COUNT":"CANTIDAD DE PERSONAL", "DEPARTAMENTO_x":"DEPARTAMENTO"}, inplace = True)

In [ ]:
df_total_2020_f.drop(columns = "DEPARTAMENTO_y", inplace = True)

In [ ]:
df_total_2020_f.drop(columns = "INDEX", inplace = True)

In [ ]:
df_total_2020_f.to_csv("/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/personal medico/consolidado_personal_4.csv", index = False, encoding = "utf-8")

In [ ]:
#condicion = (df_personal_med["DESCRIPCION_ESTABLECIMIENTO"].isin(["CORONEL PORTILLO", "FEDERICO BASADRE - YARINACOCHA",
#                                                                                                   "SISTEMA DE ATENCIÓN MÓVIL DE URGENCIAS (SAMU)", "UCAYALI"]))
#(df_personal_med["AÑO"] == 2020)&
#df_personal_med[condicion]
#df_personal_med.loc[condicion, "DESCRIPCION_ESTABLECIMIENTO"] = "CUNCHURI"

In [ ]:
#df_personal_med.to_csv("/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/personal medico/consolidado_personal.csv", index = False)

# PERSONAL MEDICO

In [ ]:
file_path_2 = "/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/personal medico/consolidado_personal.csv"
#file_name = "consolidado_personal.csv"
#total_path = os.path.join(file_path_2,file_name)
df_personal_med = pd.read_csv(file_path_2)

In [ ]:
df_dictionary = pd.read_csv("/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/diccionario de centros de salud.csv")
df_dictionary = df_dictionary[["EQUIVALENTE","CENTROS DE SALUD - COB"]]
df_dictionary

,EQUIVALENTE,CENTROS DE SALUD - COB
0,3 DE OCTUBRE,3 DE OCTUBRE
1,AGUA BLANCA,AGUA BLANCA
2,AGUAYTIA,AGUAYTIA
3,ALTO 9 DE FEBRERO,ALTO 9 DE FEBRERO
4,ALTO AGUAYTIA,ALTO AGUAYTIA
...,...,...
204,VILLA DEL CAMPO,VILLA DEL CAMPO
205,VIRGEN DEL CARMEN,VIRGEN DEL CARMEN
206,VON HUMBOLTD,VON HUMBOLTD
207,YERBAS BUENAS,YERBAS BUENAS


In [ ]:
df_dictionary["EQUIVALENTE"]= df_dictionary["EQUIVALENTE"].str.replace("  "," ")
df_dictionary["CENTROS DE SALUD - COB"] = df_dictionary["CENTROS DE SALUD - COB"].str.replace("  "," ")

## Transformacion por codigo de personal medico

In [ ]:
df_personal_med["DESCRIPCION_ESTABLECIMIENTO"] = df_personal_med["DESCRIPCION_ESTABLECIMIENTO"].str.replace(".","")
df_personal_med["DESCRIPCION_ESTABLECIMIENTO"] = df_personal_med["DESCRIPCION_ESTABLECIMIENTO"].str.replace("  "," ")

<ipython-input-7-ee190965f5ca>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_personal_med["DESCRIPCION_ESTABLECIMIENTO"] = df_personal_med["DESCRIPCION_ESTABLECIMIENTO"].str.replace(".","")


In [ ]:
df_personal_med.columns

Index(['AÑO', 'MES', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'RED',
       'MICRORRED', 'DESCRIPCION_ESTABLECIMIENTO', 'CATEGORIA', 'PCM'],
      dtype='object')

In [ ]:
df_personal_med["COUNT"] = df_personal_med["DEPARTAMENTO"]
df_personal_med = df_personal_med[['AÑO', 'MES', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'RED',
       'MICRORRED', 'DESCRIPCION_ESTABLECIMIENTO', 'CATEGORIA', "COUNT"]]
df_personal_med

,AÑO,MES,DEPARTAMENTO,PROVINCIA,DISTRITO,RED,MICRORRED,DESCRIPCION_ESTABLECIMIENTO,CATEGORIA,COUNT
0,2020,ENERO,UCAYALI,PADRE ABAD,NESHUYA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,MONTE ALEGRE-NESHUYA,I-3,UCAYALI
1,2020,ENERO,UCAYALI,PADRE ABAD,NESHUYA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,MONTE ALEGRE-NESHUYA,I-3,UCAYALI
2,2020,ENERO,UCAYALI,PADRE ABAD,NESHUYA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,MONTE ALEGRE-NESHUYA,I-3,UCAYALI
3,2020,ENERO,UCAYALI,PADRE ABAD,NESHUYA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,MONTE ALEGRE-NESHUYA,I-3,UCAYALI
4,2020,ENERO,UCAYALI,PADRE ABAD,NESHUYA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,MONTE ALEGRE-NESHUYA,I-3,UCAYALI
...,...,...,...,...,...,...,...,...,...,...
65469,2022,SETIEMBRE,UCAYALI,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,SAN JOSE DE YARINACOCHA,CLAS SAN PABLO DE TUSHMO,I-2,UCAYALI
65470,2022,SETIEMBRE,UCAYALI,CORONEL PORTILLO,CAMPOVERDE,FEDERICO BASADRE - YARINACOCHA,CAMPO VERDE,PUESTO DE SALUD ALTO MANANTAY,I-1,UCAYALI
65471,2022,SETIEMBRE,UCAYALI,CORONEL PORTILLO,IPARIA,CORONEL PORTILLO,IPARIA,NUEVO AHUAYPA,I-1,UCAYALI
65472,2022,SETIEMBRE,UCAYALI,CORONEL PORTILLO,CALLERIA,CORONEL PORTILLO,SAN FERNANDO,CALLERIA,I-1,UCAYALI


In [ ]:
#CAMBIO DE PUERTO NUEVO A PS PUERTO NUEVO
condicion = (df_personal_med["DESCRIPCION_ESTABLECIMIENTO"] == "PUERTO NUEVO") & (df_personal_med["MICRORRED"] == "IPARIA")
df_personal_med.loc[condicion, "DESCRIPCION_ESTABLECIMIENTO"] = "PS PUERTO NUEVO"

In [ ]:
condicion = (df_personal_med["DESCRIPCION_ESTABLECIMIENTO"] == "PUERTO NUEVO") & (df_personal_med["MICRORRED"] == "CORONEL PORTILLO") & (df_personal_med["RED"] == "CORONEL PORTILLO") & (df_personal_med["MES"] == "OCTUBRE")
df_personal_med.loc[condicion, "DESCRIPCION_ESTABLECIMIENTO"] = "PS PUERTO NUEVO"
df_personal_med.loc[condicion, "MICRORRED"] = "IPARIA"

In [ ]:
condicion = (df_personal_med["DESCRIPCION_ESTABLECIMIENTO"] == "PUERTO NUEVO") & (df_personal_med["MICRORRED"] == "AGUAYTIA") & (df_personal_med["RED"] == "AGUAYTIA") & (df_personal_med["MES"] == "OCTUBRE")
df_personal_med.loc[condicion, "DESCRIPCION_ESTABLECIMIENTO"] = "PUERTO NUEVO"
df_personal_med.loc[condicion, "MICRORRED"] = "SAN ALEJANDRO"

In [ ]:
#PS TUPAC AMARU KM 15
condicion = (df_personal_med["DESCRIPCION_ESTABLECIMIENTO"] == "TUPAC AMARU") & (df_personal_med["MICRORRED"] == "SAN FERNANDO") & (df_personal_med["RED"] == "CORONEL PORTILLO")
df_personal_med.loc[condicion, "DESCRIPCION_ESTABLECIMIENTO"] = "PS TUPAC AMARU KM 15"

In [ ]:
df_personal_med.to_csv("/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/personal medico/consolidado_personal_2.csv", index = False)

## After pointbreak

In [ ]:
file_path_2 = "/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/personal medico/consolidado_personal_3.csv"
#file_name = "consolidado_personal.csv"
#total_path = os.path.join(file_path_2,file_name)
df_personal_med = pd.read_csv(file_path_2)

In [ ]:
dict_eq = dict(zip(df_dictionary['EQUIVALENTE'], df_dictionary['CENTROS DE SALUD - COB']))
df_personal_med.replace({"DESCRIPCION_ESTABLECIMIENTO": dict_eq},inplace=True)
#df_personal_med['DESCRIPCION_ESTABLECIMIENTO'] = df_personal_med['DESCRIPCION_ESTABLECIMIENTO'].replace(".","")
df_personal_med

,AÑO,MES,DEPARTAMENTO,PROVINCIA,DISTRITO,RED,MICRORRED,DESCRIPCION_ESTABLECIMIENTO,CATEGORIA,COUNT
0,2020,ENERO,UCAYALI,PADRE ABAD,NESHUYA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,MONTE ALEGRENESHUYA,I-3,UCAYALI
1,2020,ENERO,UCAYALI,PADRE ABAD,NESHUYA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,MONTE ALEGRENESHUYA,I-3,UCAYALI
2,2020,ENERO,UCAYALI,PADRE ABAD,NESHUYA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,MONTE ALEGRENESHUYA,I-3,UCAYALI
3,2020,ENERO,UCAYALI,PADRE ABAD,NESHUYA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,MONTE ALEGRENESHUYA,I-3,UCAYALI
4,2020,ENERO,UCAYALI,PADRE ABAD,NESHUYA,FEDERICO BASADRE - YARINACOCHA,MONTE ALEGRE-CURIMANA,MONTE ALEGRENESHUYA,I-3,UCAYALI
...,...,...,...,...,...,...,...,...,...,...
65469,2022,SETIEMBRE,UCAYALI,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,SAN JOSE DE YARINACOCHA,CLAS SAN PABLO DE TUSHMO,I-2,UCAYALI
65470,2022,SETIEMBRE,UCAYALI,CORONEL PORTILLO,CAMPOVERDE,FEDERICO BASADRE - YARINACOCHA,CAMPO VERDE,PUESTO DE SALUD ALTO MANANTAY,I-1,UCAYALI
65471,2022,SETIEMBRE,UCAYALI,CORONEL PORTILLO,IPARIA,CORONEL PORTILLO,IPARIA,PS NUEVO AHUAYPA,I-1,UCAYALI
65472,2022,SETIEMBRE,UCAYALI,CORONEL PORTILLO,CALLERIA,CORONEL PORTILLO,SAN FERNANDO,PS CALLERIA,I-1,UCAYALI


In [ ]:
df_personal_med.drop(columns = "CATEGORIA", inplace = True)

In [ ]:
df_summary = df_personal_med.groupby(['AÑO', 'MES', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'RED',
       'MICRORRED', 'DESCRIPCION_ESTABLECIMIENTO']).agg(pd.Series.count)
df_summary

COUNT
AÑO  MES       DEPARTAMENTO PROVINCIA DISTRITO RED              MICRORRED DESCRIPCION_ESTABLECIMIENTO       
2020 ABRIL     UCAYALI      ATALAYA   RAIMONDI ATALAYA          ATALAYA   I1 00005503 MALDONADILLO         5
                                                                          I1 00005504 COCANI               1
                                                                          I1 00005506 RAMON CASTILLA       2
                                                                          I1 00005507 BOCA PUCANI          1
                                                                          I1 00005508 UNINI                3
...                                                                                                      ...
2022 SETIEMBRE UCAYALI      PURUS     PURUS    CORONEL PORTILLO PURUS     CS PURUS                        13
                                                                          PS BALTA                         1
                                                                          PS MIGUEL GRAU                   1
                                                                          PS SAN BERNARDO                  1
                                                                          PS SAN MARCOS                    1

[6509 rows x 1 columns]

In [ ]:
df_summary.reset_index(inplace = True)
df_summary_2022 = df_summary[df_summary["AÑO"] == 2022]
df_summary[df_summary["AÑO"] == 2022]["DESCRIPCION_ESTABLECIMIENTO"].nunique()

215

In [ ]:
df_summary_2022

,index,AÑO,MES,DEPARTAMENTO,PROVINCIA,DISTRITO,RED,MICRORRED,DESCRIPCION_ESTABLECIMIENTO,COUNT
4688,4688,2022,ABRIL,UCAYALI,ATALAYA,RAIMONDI,ATALAYA,ATALAYA,I1 00005503 MALDONADILLO,3
4689,4689,2022,ABRIL,UCAYALI,ATALAYA,RAIMONDI,ATALAYA,ATALAYA,I1 00005504 COCANI,1
4690,4690,2022,ABRIL,UCAYALI,ATALAYA,RAIMONDI,ATALAYA,ATALAYA,I1 00005506 RAMON CASTILLA,2
4691,4691,2022,ABRIL,UCAYALI,ATALAYA,RAIMONDI,ATALAYA,ATALAYA,I1 00005507 BOCA PUCANI,1
4692,4692,2022,ABRIL,UCAYALI,ATALAYA,RAIMONDI,ATALAYA,ATALAYA,I1 00005508 UNINI,2
...,...,...,...,...,...,...,...,...,...,...
6504,6504,2022,SETIEMBRE,UCAYALI,PURUS,PURUS,CORONEL PORTILLO,PURUS,CS PURUS,13
6505,6505,2022,SETIEMBRE,UCAYALI,PURUS,PURUS,CORONEL PORTILLO,PURUS,PS BALTA,1
6506,6506,2022,SETIEMBRE,UCAYALI,PURUS,PURUS,CORONEL PORTILLO,PURUS,PS MIGUEL GRAU,1
6507,6507,2022,SETIEMBRE,UCAYALI,PURUS,PURUS,CORONEL PORTILLO,PURUS,PS SAN BERNARDO,1


In [ ]:
df_summary_2022.to_csv("/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/personal medico/df_summary_2022.csv")

In [ ]:
df_revision = df_summary_2022.groupby(by =['AÑO', 'MES','DESCRIPCION_ESTABLECIMIENTO']).agg(pd.Series.count)
df_revision[df_revision["COUNT"]>1]

index  DEPARTAMENTO  PROVINCIA  \
AÑO  MES       DESCRIPCION_ESTABLECIMIENTO                                   
2022 ABRIL     I1 00005522 TUPAC AMARU          2             2          2   
     AGOSTO    I1 00005522 TUPAC AMARU          2             2          2   
     JULIO     I1 00005522 TUPAC AMARU          2             2          2   
     JUNIO     I1 00005522 TUPAC AMARU          2             2          2   
     MAYO      I1 00005522 TUPAC AMARU          2             2          2   
     SETIEMBRE I1 00005522 TUPAC AMARU          2             2          2   

                                            DISTRITO  RED  MICRORRED  COUNT  
AÑO  MES       DESCRIPCION_ESTABLECIMIENTO                                   
2022 ABRIL     I1 00005522 TUPAC AMARU             2    2          2      2  
     AGOSTO    I1 00005522 TUPAC AMARU             2    2          2      2  
     JULIO     I1 00005522 TUPAC AMARU             2    2          2      2  
     JUNIO     I1 00005522 TUPAC AMARU             2    2          2      2  
     MAYO      I1 00005522 TUPAC AMARU             2    2          2      2  
     SETIEMBRE I1 00005522 TUPAC AMARU             2    2          2      2

# 2022 merge

In [ ]:
cob_2022 = pd.read_csv("/content/cob_formateado_2022.csv")
cob_2022

,CENTRO_SALUD,ANIO,DEPARTAMENTO,RANGO_EDAD,EFERMERDAD,MES,%,CANTIDAD
0,CENTRO AMERICA,2022,UCAYALI,1_ANIO,AMA,ABRIL,2.132701422,9.0
1,CENTRO AMERICA,2022,UCAYALI,1_ANIO,AMA,AGOSTO,3.08056872,13.0
2,CENTRO AMERICA,2022,UCAYALI,1_ANIO,AMA,DICIEMBRE,1.658767773,7.0
3,CENTRO AMERICA,2022,UCAYALI,1_ANIO,AMA,ENERO,4.265402844,18.0
4,CENTRO AMERICA,2022,UCAYALI,1_ANIO,AMA,FEBRERO,1.658767773,7.0
...,...,...,...,...,...,...,...,...
38695,HOSPITAL AMAZONICO YARINACOCHA,2022,UCAYALI,RN,HVB,MARZO,0,289.0
38696,HOSPITAL AMAZONICO YARINACOCHA,2022,UCAYALI,RN,HVB,MAYO,0,292.0
38697,HOSPITAL AMAZONICO YARINACOCHA,2022,UCAYALI,RN,HVB,NOVIEMBRE,0,300.0
38698,HOSPITAL AMAZONICO YARINACOCHA,2022,UCAYALI,RN,HVB,OCTUBRE,0,274.0


In [ ]:
df_summary_2022 = pd.read_csv("/content/df_summary_2022.csv")
df_summary_2022

,ANIO,MES,DEPARTAMENTO,PROVINCIA,DISTRITO,RED,MICRORRED,DESCRIPCION_ESTABLECIMIENTO,COUNT
0,2022,ABRIL,UCAYALI,ATALAYA,RAIMONDI,ATALAYA,ATALAYA,I1 00005503 MALDONADILLO,3
1,2022,ABRIL,UCAYALI,ATALAYA,RAIMONDI,ATALAYA,ATALAYA,I1 00005504 COCANI,1
2,2022,ABRIL,UCAYALI,ATALAYA,RAIMONDI,ATALAYA,ATALAYA,I1 00005506 RAMON CASTILLA,2
3,2022,ABRIL,UCAYALI,ATALAYA,RAIMONDI,ATALAYA,ATALAYA,I1 00005507 BOCA PUCANI,1
4,2022,ABRIL,UCAYALI,ATALAYA,RAIMONDI,ATALAYA,ATALAYA,I1 00005508 UNINI,2
...,...,...,...,...,...,...,...,...,...
1816,2022,SETIEMBRE,UCAYALI,PURUS,PURUS,CORONEL PORTILLO,PURUS,CS PURUS,13
1817,2022,SETIEMBRE,UCAYALI,PURUS,PURUS,CORONEL PORTILLO,PURUS,PS BALTA,1
1818,2022,SETIEMBRE,UCAYALI,PURUS,PURUS,CORONEL PORTILLO,PURUS,PS MIGUEL GRAU,1
1819,2022,SETIEMBRE,UCAYALI,PURUS,PURUS,CORONEL PORTILLO,PURUS,PS SAN BERNARDO,1


In [ ]:
df_final_2022 = cob_2022.merge(df_summary_2022, left_on=["ANIO","MES","CENTRO_SALUD"], right_on=["ANIO","MES", "DESCRIPCION_ESTABLECIMIENTO"], how = "left")
df_final_2022

,CENTRO_SALUD,ANIO,DEPARTAMENTO_x,RANGO_EDAD,EFERMERDAD,MES,%,CANTIDAD,DEPARTAMENTO_y,PROVINCIA,DISTRITO,RED,MICRORRED,DESCRIPCION_ESTABLECIMIENTO,COUNT
0,CENTRO AMERICA,2022,UCAYALI,1_ANIO,AMA,ABRIL,2.132701422,9.0,UCAYALI,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,CENTRO AMERICA,28.0
1,CENTRO AMERICA,2022,UCAYALI,1_ANIO,AMA,AGOSTO,3.08056872,13.0,UCAYALI,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,CENTRO AMERICA,31.0
2,CENTRO AMERICA,2022,UCAYALI,1_ANIO,AMA,DICIEMBRE,1.658767773,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CENTRO AMERICA,2022,UCAYALI,1_ANIO,AMA,ENERO,4.265402844,18.0,UCAYALI,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,CENTRO AMERICA,28.0
4,CENTRO AMERICA,2022,UCAYALI,1_ANIO,AMA,FEBRERO,1.658767773,7.0,UCAYALI,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,CENTRO AMERICA,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38695,HOSPITAL AMAZONICO YARINACOCHA,2022,UCAYALI,RN,HVB,MARZO,0,289.0,UCAYALI,CORONEL PORTILLO,YARINACOCHA,NO PERTENECE A NINGUNA RED,NO PERTENECE A NINGUNA MICRORED,HOSPITAL AMAZONICO YARINACOCHA,360.0
38696,HOSPITAL AMAZONICO YARINACOCHA,2022,UCAYALI,RN,HVB,MAYO,0,292.0,UCAYALI,CORONEL PORTILLO,YARINACOCHA,NO PERTENECE A NINGUNA RED,NO PERTENECE A NINGUNA MICRORED,HOSPITAL AMAZONICO YARINACOCHA,360.0
38697,HOSPITAL AMAZONICO YARINACOCHA,2022,UCAYALI,RN,HVB,NOVIEMBRE,0,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38698,HOSPITAL AMAZONICO YARINACOCHA,2022,UCAYALI,RN,HVB,OCTUBRE,0,274.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_test = df_final_2022.groupby("CENTRO_SALUD")["MICRORRED"].nunique().reset_index(name='DistinctValuesCount')
df_test[df_test["DistinctValuesCount"]>1]

,CENTRO_SALUD,DistinctValuesCount
19,CS BELLAVISTA,2
94,PS ABUJAO,2
102,PS CHANCAY,2
108,PS DULCE GLORIA,2
152,PS SANTA ELENA,2
155,PS SANTA ROSA DE ABUJAO,2
164,PS TUPAC AMARU KM 15,2
169,PS YANAMAYO,2


In [ ]:
df_final_2022['DISTRITO'] =  df_final_2022.groupby('CENTRO_SALUD')['DISTRITO'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)

In [ ]:
df_final_2022['PROVINCIA'] =  df_final_2022.groupby('CENTRO_SALUD')['PROVINCIA'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)

In [ ]:
condicion = df_final_2022["CENTRO_SALUD"] == "CS BELLAVISTA"
df_final_2022.loc[condicion, "MICRORRED"] = "09 DE OCTUBRE"

condicion_1 = df_final_2022["CENTRO_SALUD"] == "PS CHANCAY"
df_final_2022.loc[condicion_1, "MICRORRED"] = "09 DE OCTUBRE"

condicion_2 = df_final_2022["CENTRO_SALUD"] == "PS DULCE GLORIA"
df_final_2022.loc[condicion_2, "MICRORRED"] = "YURUA"

condicion_3 = df_final_2022["CENTRO_SALUD"] == "PS ABUJAO"
df_final_2022.loc[condicion_3, "MICRORRED"] = "SAN FERNANDO"

condicion_4 = df_final_2022["CENTRO_SALUD"] == "PS SANTA ELENA"
df_final_2022.loc[condicion_4, "MICRORRED"] = "09 DE OCTUBRE"

condicion_5 = df_final_2022["CENTRO_SALUD"] == "PS SANTA ROSA DE ABUJAO"
df_final_2022.loc[condicion_5, "MICRORRED"] = "SAN FERNANDO"

condicion_6 = df_final_2022["CENTRO_SALUD"] == "PS TUPAC AMARU KM 15"
df_final_2022.loc[condicion_6, "MICRORRED"] = "09 DE OCTUBRE"

condicion_7 = df_final_2022["CENTRO_SALUD"] == "PS YANAMAYO"
df_final_2022.loc[condicion_7, "MICRORRED"] = "09 DE OCTUBRE"

In [ ]:
df_test = df_final_2022.groupby("CENTRO_SALUD")["MICRORRED"].nunique().reset_index(name='DistinctValuesCount')
df_test[df_test["DistinctValuesCount"]>1]

,CENTRO_SALUD,DistinctValuesCount


In [ ]:
df_final_2022['RED'] =  df_final_2022.groupby('CENTRO_SALUD')['RED'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)

In [ ]:
df_final_2022['MICRORRED'] =  df_final_2022.groupby('CENTRO_SALUD')['MICRORRED'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)

In [ ]:
df_final_2022

,CENTRO_SALUD,ANIO,DEPARTAMENTO_x,RANGO_EDAD,EFERMERDAD,MES,%,CANTIDAD,DEPARTAMENTO_y,PROVINCIA,DISTRITO,RED,MICRORRED,DESCRIPCION_ESTABLECIMIENTO,COUNT
0,CENTRO AMERICA,2022,UCAYALI,1_ANIO,AMA,ABRIL,2.132701422,9.0,UCAYALI,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,CENTRO AMERICA,28.0
1,CENTRO AMERICA,2022,UCAYALI,1_ANIO,AMA,AGOSTO,3.08056872,13.0,UCAYALI,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,CENTRO AMERICA,31.0
2,CENTRO AMERICA,2022,UCAYALI,1_ANIO,AMA,DICIEMBRE,1.658767773,7.0,NaN,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,NaN,NaN
3,CENTRO AMERICA,2022,UCAYALI,1_ANIO,AMA,ENERO,4.265402844,18.0,UCAYALI,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,CENTRO AMERICA,28.0
4,CENTRO AMERICA,2022,UCAYALI,1_ANIO,AMA,FEBRERO,1.658767773,7.0,UCAYALI,CORONEL PORTILLO,YARINACOCHA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,CENTRO AMERICA,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38695,HOSPITAL AMAZONICO YARINACOCHA,2022,UCAYALI,RN,HVB,MARZO,0,289.0,UCAYALI,CORONEL PORTILLO,YARINACOCHA,NO PERTENECE A NINGUNA RED,NO PERTENECE A NINGUNA MICRORED,HOSPITAL AMAZONICO YARINACOCHA,360.0
38696,HOSPITAL AMAZONICO YARINACOCHA,2022,UCAYALI,RN,HVB,MAYO,0,292.0,UCAYALI,CORONEL PORTILLO,YARINACOCHA,NO PERTENECE A NINGUNA RED,NO PERTENECE A NINGUNA MICRORED,HOSPITAL AMAZONICO YARINACOCHA,360.0
38697,HOSPITAL AMAZONICO YARINACOCHA,2022,UCAYALI,RN,HVB,NOVIEMBRE,0,300.0,NaN,CORONEL PORTILLO,YARINACOCHA,NO PERTENECE A NINGUNA RED,NO PERTENECE A NINGUNA MICRORED,NaN,NaN
38698,HOSPITAL AMAZONICO YARINACOCHA,2022,UCAYALI,RN,HVB,OCTUBRE,0,274.0,NaN,CORONEL PORTILLO,YARINACOCHA,NO PERTENECE A NINGUNA RED,NO PERTENECE A NINGUNA MICRORED,NaN,NaN


In [ ]:
df_final_2022.to_csv("/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/Coberturas_Ucayali/df_2022.csv", index = False)

# Preprocessing

In [ ]:
def personal_processing(df_cobertura, filename, dict_mes, year):
  df_personal = df_cobertura[["UBIGEO","DEPARTAMENTO","PROVINCIA","DISTRITO","RED","MICRORRED","DESCRIPCION ESTABLECIMIENTO","CALSIFICACION","CATEGORIA","profesion11"]]
  condicion_region = df_personal["DEPARTAMENTO"] == 'UCAYALI'
  condicion_profesion = df_personal["profesion11"].isin(["TECNICO EN ENFERMERIA","ENFERMERA(O)","ENFERMERIA"])
  df_personal = df_personal[condicion_region & condicion_profesion]
  df_personal["MES"] = dict_mes[int(filename[:2])]
  df_personal["AÑO"] = int(year)
  df_agrupado = df_personal.groupby(['AÑO', 'MES','UBIGEO', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'RED', 'MICRORRED', 'DESCRIPCION ESTABLECIMIENTO', 'CATEGORIA'], as_index=False).agg(pd.Series.count)
  return df_agrupado[['AÑO', 'MES','UBIGEO', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'RED', 'MICRORRED', 'DESCRIPCION ESTABLECIMIENTO']]

In [ ]:
sheet_name = "Hoja1"
dict_mes = {
    1: "ENERO",
    2: "FEBRERO",
    3: "MARZO",
    4: "ABRIL",
    5: "MAYO",
    6: "JUNIO",
    7: "JULIO",
    8: "AGOSTO",
    9: "SEPTIEMBRE",
    10: "OCTUBRE",
    11: "NOVIEMBRE",
    12: "DICIEMBRE",
}

In [ ]:
años = ["2020", "2021", "2022"]
df_agrupado_list = []
for year in años:
  print("="*40)
  print(year)
  print("="*40)
  file_path_2 = "/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/personal medico/"
  for file_xslx in os.listdir(file_path_2+year)[:36]:
    print(file_xslx)
    total_path = os.path.join(file_path_2, year, file_xslx)
    wb = xl.load_workbook(total_path, read_only=True)
    df_cobertura = pd.read_excel(total_path, sheet_name = wb.sheetnames[1], skiprows=3)
    df_agrupado = personal_processing(df_cobertura, file_xslx, dict_mes, year)
    df_agrupado_list.append(df_agrupado)
df_personal_final = pd.concat(df_agrupado_list)

In [ ]:
file_xslx

'01_ENERO_2020.xlsx'

In [ ]:
filename = "01_ENERO_2020.xlsx"
year = "2020"
file_path_2 = "/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/personal medico/2020/"
sheet_name = "Hoja1"

In [ ]:
df_cobertura = pd.read_excel(file_path_2+filename, sheet_name=sheet_name, skiprows=3)

In [ ]:
dict_mes[int(filename[:2])]

'ENERO'

In [ ]:
df_personal = df_cobertura[["UBIGEO","DEPARTAMENTO","PROVINCIA","DISTRITO","RED","MICRORRED","DESCRIPCION ESTABLECIMIENTO","CALSIFICACION","CATEGORIA","cargo"]]

NameError: name 'df_cobertura' is not defined

In [ ]:
#CONDITIONS
condicion_region = df_personal["DEPARTAMENTO"] == 'UCAYALI'
condicion_profesion = df_personal["profesion11"].isin(["TECNICO EN ENFERMERIA","ENFERMERA(O)","ENFERMERIA"])
df_personal = df_personal[condicion_region & condicion_profesion]
df_personal["MES"] = dict_mes[int(filename[:2])]
df_personal["AÑO"] = int(year)
df_agrupado = df_personal.groupby(['AÑO', 'MES','UBIGEO', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'RED', 'MICRORRED', 'DESCRIPCION ESTABLECIMIENTO', 'CATEGORIA'], as_index=False).agg(pd.Series.count)
df_agrupado = df_agrupado[['AÑO', 'MES','UBIGEO', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'RED', 'MICRORRED', 'DESCRIPCION ESTABLECIMIENTO']]
df_agrupado[df_agrupado.duplicated()]

,AÑO,MES,UBIGEO,DEPARTAMENTO,PROVINCIA,DISTRITO,RED,MICRORRED,DESCRIPCION ESTABLECIMIENTO


In [ ]:
df_agrupado[(df_agrupado['DESCRIPCION ESTABLECIMIENTO'] == 'NUEVO PARAISO') & (df_agrupado['MICRORRED'] == 'MASISEA')]['DESCRIPCION ESTABLECIMIENTO'] = "NUEVO PARAISO MASISEA"
df_agrupado[(df_agrupado['DESCRIPCION ESTABLECIMIENTO'] == 'NUEVO PARAISO')]

<ipython-input-71-43941ca4f6a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agrupado[(df_agrupado['DESCRIPCION ESTABLECIMIENTO'] == 'NUEVO PARAISO') & (df_agrupado['MICRORRED'] == 'MASISEA')]['DESCRIPCION ESTABLECIMIENTO'] = "NUEVO PARAISO MASISEA"


,AÑO,MES,UBIGEO,DEPARTAMENTO,PROVINCIA,DISTRITO,RED,MICRORRED,DESCRIPCION ESTABLECIMIENTO,CALSIFICACION,CATEGORIA,profesion11
27,2019,ENERO,250101,UCAYALI,CORONEL PORTILLO,CALLERIA,FEDERICO BASADRE - YARINACOCHA,NUEVO PARAISO,NUEVO PARAISO,CENTROS DE SALUD O CENTROS MEDICOS,I-3,25
75,2019,ENERO,250104,UCAYALI,CORONEL PORTILLO,MASISEA,CORONEL PORTILLO,MASISEA,NUEVO PARAISO,PUESTOS DE SALUD O POSTAS DE SALUD,I-1,1


In [ ]:
df_test_2020['ESTABLECIMIENTO_DE_SALUD'].unique()

array(['CENTRO AMERICA', 'CLAS TUPAC AMARU', 'DOS DE MAYO',
       'HUSARES DEL PERU', 'JOSE OLAYA', 'NUEVO PARAISO', 'SANIDAD AEREA',
       'SANTA TERESITA', 'SHIRAMBARI', 'CASHIVOCOCHA',
       'CLAS  BELLAVISTA DE YARINACOCH', 'CLAS DE SAN JUAN YARINACOCHA',
       'CLAS SAN PABLO DE TUSHMO', 'CLAS UNION ZAPOTILLO',
       'ESPERANZA DE PANAILLO', 'HUITOCOCHA', 'LEONCIO PRADO',
       'MARISCAL SUCRE', 'NUEVA ALEJANDRIA', 'NUEVA LUZ DE FATIMA',
       'NUEVA UNION', 'SAN FRANCISCO DE YARINACOCHA',
       'SAN JOSE DE YARINACOCHA', 'SANTA ROSA', 'AGUA BLANCA',
       'ALTO 9 DE FEBRERO', 'ANTONIO RAYMONDI', 'CAMPO VERDE',
       'EL PORVENIR KM 25', 'LA MERCED DE NESHUYA', 'LA VICTORIA',
       'LOS PINOS', 'NUEVA ESPERANZA', 'NUEVA TUNUYA',
       'PS LAS MERCEDES KM 42', 'PIMENTAL', 'PUEBLO LIBRE',
       'PUESTO DE SALUD ALTO MANANTAY', 'PUESTO DE SALUD AMAKELLA',
       'SAN JOSE KM 26', 'SAN MARTIN DE MOJARAL', 'SAN PEDRO KM 47',
       'SANTA ROSA KM 50 CFB', 'SEOR DE LOS MILA

# RED - MICRORRED - CENTRO DE SALUD CHECK

In [ ]:
base_ds_2020 = pd.read_csv("/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/personal medico/consolidado_personal_5.csv", sep = ";")
base_ds_2020_info = base_ds_2020 [["RED","MICRORRED" ,"CENTRO_SALUD"]]

In [ ]:
base_ds_2020_info[base_ds_2020_info["RED"].isnull()].groupby()

,RED,MICRORRED,CENTRO_SALUD
1800,NaN,NaN,CLAS BELLAVISTA DE YARINACOCH
1801,NaN,NaN,CLAS BELLAVISTA DE YARINACOCH
1802,NaN,NaN,CLAS BELLAVISTA DE YARINACOCH
1803,NaN,NaN,CLAS BELLAVISTA DE YARINACOCH
1804,NaN,NaN,CLAS BELLAVISTA DE YARINACOCH
...,...,...,...
37255,NaN,NaN,HOSPITAL II PUCALLPA ESSALUD
37256,NaN,NaN,HOSPITAL II PUCALLPA ESSALUD
37257,NaN,NaN,HOSPITAL II PUCALLPA ESSALUD
37258,NaN,NaN,HOSPITAL II PUCALLPA ESSALUD


#Borrador

## Centros de Salud Preprocessing - 2020

In [ ]:
df_centros_de_salud = df_cobertura[df_cobertura['Marca'] != 'x'][['ESTABLECIMIENTO DE SALUD']]
df_centros_de_salud.dropna(inplace = True)
df_centros_de_salud

,ESTABLECIMIENTO DE SALUD
3,CENTRO AMERICA
4,CLAS TUPAC AMARU
5,DOS DE MAYO
6,HUSARES DEL PERU.
7,JOSE OLAYA
...,...
226,SAN PEDRO DE CHIO
227,SINCHI ROCA
229,HOSPITAL II PUCALLPA- ESSALUD
230,HOSPITAL REGIONAL DE PUCALLPA


In [ ]:
df_centros_de_salud.dropna(inplace = True)
df_centros_de_salud['ESTABLECIMIENTO DE SALUD'] = df_centros_de_salud['ESTABLECIMIENTO DE SALUD'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
df_centros_de_salud['ESTABLECIMIENTO DE SALUD'] = df_centros_de_salud['ESTABLECIMIENTO DE SALUD'].str.strip()
df_centros_de_salud['ESTABLECIMIENTO DE SALUD'] = df_centros_de_salud['ESTABLECIMIENTO DE SALUD'].str.replace('    ',' ')
df_centros_de_salud['ESTABLECIMIENTO DE SALUD'] = df_centros_de_salud['ESTABLECIMIENTO DE SALUD'].str.replace('  ',' ')
df_centros_de_salud.rename(columns={'ESTABLECIMIENTO DE SALUD': 'ESTABLECIMIENTO_DE_SALUD'}, inplace=True)

In [ ]:
# agregando el año
df_centros_de_salud['AÑO'] = 2020
df_centros_de_salud

,ESTABLECIMIENTO_DE_SALUD,AÑO
3,CENTRO AMERICA,2020
4,CLAS TUPAC AMARU,2020
5,DOS DE MAYO,2020
6,HUSARES DEL PERU,2020
7,JOSE OLAYA,2020
...,...,...
226,SAN PEDRO DE CHIO,2020
227,SINCHI ROCA,2020
229,HOSPITAL II PUCALLPA ESSALUD,2020
230,HOSPITAL REGIONAL DE PUCALLPA,2020


## Recien Nacidos - BCG

In [ ]:
df_rn_BCG = df_cobertura.iloc[:, 3:27]
df_rn_BCG.columns = df_rn_BCG.iloc[0]
df_rn_BCG = df_rn_BCG[1:]
df_rn_BCG

,ENE,%,FEB,%,MAR,%,ABR,%,MAY,%,...,AGO,%,SEPT,%,OCT,%,NOV,%,DIC,%
1,55,2.442274,59,2.619893,53,2.353464,124,5.506217,72,3.197158,...,150,6.660746,145,6.438721,131,5.817052,102,4.529307,110,4.884547
2,24,1.525747,21,1.335029,17,1.080737,57,3.623649,6,0.381437,...,85,5.403687,93,5.91227,72,4.577241,46,2.924348,58,3.687222
3,2,0.505051,3,0.757576,1,0.252525,0,0,0,0,...,0,0,1,0.252525,0,0,0,0,1,0.252525
4,1,0.273973,0,0,0,0,0,0,0,0,...,0,0,2,0.547945,0,0,1,0.273973,1,0.273973
5,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,542,0,575,0,587,0,315,0,265,0,...,389,0,424,0,445,0,437,0,399,0
229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def define_multilevel_col(df, enfermedad, rango_de_edad):
  month_cols = pd.MultiIndex.from_tuples([
                                    (f"{rango_de_edad}",f"{enfermedad}","ENERO", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","ENERO", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","FEBRERO", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","FEBRERO", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","MARZO", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","MARZO", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","ABRIL", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","ABRIL", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","MAYO", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","MAYO", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","JUNIO", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","JUNIO", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","JULIO", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","JULIO", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","AGOSTO", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","AGOSTO", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","SETIEMBRE", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","SETIEMBRE", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","OCTUBRE", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","OCTUBRE", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","NOVIEMBRE", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","NOVIEMBRE", "%"),
                                    (f"{rango_de_edad}",f"{enfermedad}","DICIEMBRE", "CANTIDAD"),(f"{rango_de_edad}",f"{enfermedad}","DICIEMBRE", "%")])
  return pd.DataFrame(df.values, columns=month_cols)

In [ ]:
enfermedad = 'BCG'
rango_de_edad = 'RN'
df_rn_BCG_final = define_multilevel_col(df_rn_BCG,enfermedad, rango_de_edad)
df_rn_BCG_stacked = df_rn_BCG_final.stack(level = [0,1,2])
df_rn_BCG_stacked

% CANTIDAD
0   RN BCG ABRIL       5.506217      124
           AGOSTO      6.660746      150
           DICIEMBRE   4.884547      110
           ENERO       2.442274       55
           FEBRERO     2.619893       59
...                         ...      ...
231 RN BCG MARZO      16.694737      793
           MAYO       10.568421      502
           NOVIEMBRE  16.357895      777
           OCTUBRE    17.452632      829
           SETIEMBRE  16.631579      790

[2784 rows x 2 columns]

In [ ]:
df_rn_BCG_final

RN                                                              \
         BCG                                                               
       ENERO             FEBRERO               MARZO               ABRIL   
    CANTIDAD          % CANTIDAD          % CANTIDAD          % CANTIDAD   
0         55   2.442274       59   2.619893       53   2.353464      124   
1         24   1.525747       21   1.335029       17   1.080737       57   
2          2   0.505051        3   0.757576        1   0.252525        0   
3          1   0.273973        0          0        0          0        0   
4          0          0        0          0        0          0        0   
..       ...        ...      ...        ...      ...        ...      ...   
227      542          0      575          0      587          0      315   
228        0          0        0          0        0          0        0   
229        0          0        0          0        0          0        0   
230        0          0        0          0        0          0        0   
231      783  16.484211      788  16.589474      793  16.694737      628   

                                    ...                                \
                                    ...                                 
                   MAYO             ...   AGOSTO            SETIEMBRE   
             % CANTIDAD          %  ... CANTIDAD          %  CANTIDAD   
0     5.506217       72   3.197158  ...      150   6.660746       145   
1     3.623649        6   0.381437  ...       85   5.403687        93   
2            0        0          0  ...        0          0         1   
3            0        0          0  ...        0          0         2   
4            0        0          0  ...        1          0         0   
..         ...      ...        ...  ...      ...        ...       ...   
227          0      265          0  ...      389          0       424   
228          0        0          0  ...        0          0         0   
229          0        0          0  ...        0          0         0   
230          0        0          0  ...        0          0         0   
231  13.221053      502  10.568421  ...      773  16.273684       790   

                                                                              
                                                                              
                OCTUBRE            NOVIEMBRE            DICIEMBRE             
             % CANTIDAD          %  CANTIDAD          %  CANTIDAD          %  
0     6.438721      131   5.817052       102   4.529307       110   4.884547  
1      5.91227       72   4.577241        46   2.924348        58   3.687222  
2     0.252525        0          0         0          0         1   0.252525  
3     0.547945        0          0         1   0.273973         1   0.273973  
4            0        0          0         0          0         0          0  
..         ...      ...        ...       ...        ...       ...        ...  
227          0      445          0       437          0       399          0  
228          0        0          0         0          0         0          0  
229          0        0          0         0          0         0          0  
230          0        0          0         0          0         0          0  
231  16.631579      829  17.452632       777  16.357895       748  15.747368  

[232 rows x 24 columns]

In [ ]:
preprocessing_enf_edad(df_cobertura, 3)

,ENE,%,FEB,%,MAR,%,ABR,%,MAY,%,...,AGO,%,SEPT,%,OCT,%,NOV,%,DIC,%
1,55,2.442274,59,2.619893,53,2.353464,124,5.506217,72,3.197158,...,150,6.660746,145,6.438721,131,5.817052,102,4.529307,110,4.884547
2,24,1.525747,21,1.335029,17,1.080737,57,3.623649,6,0.381437,...,85,5.403687,93,5.91227,72,4.577241,46,2.924348,58,3.687222
3,2,0.505051,3,0.757576,1,0.252525,0,0,0,0,...,0,0,1,0.252525,0,0,0,0,1,0.252525
4,1,0.273973,0,0,0,0,0,0,0,0,...,0,0,2,0.547945,0,0,1,0.273973,1,0.273973
5,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,542,0,575,0,587,0,315,0,265,0,...,389,0,424,0,445,0,437,0,399,0
229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Recien Nacidos - HVB


In [ ]:
df_rn_HVB = df_cobertura.iloc[:, 29:53]
df_rn_HVB.columns = df_rn_HVB.iloc[0]
df_rn_HVB = df_rn_HVB[1:]
df_rn_HVB

,ENE,%,FEB,%,MAR,%,ABR,%,MAY,%,...,AGO,%,SEPT,%,OCT,%,NOV,%,DIC,%
1,64,117.272579,64,2.841918,57,2.531083,127,5.639432,106,4.706927,...,158,7.015986,155,6.882771,137,6.083481,106,4.706927,108,4.795737
2,28,1.780038,22,1.398601,17,1.080737,57,3.623649,35,2.225048,...,91,5.785124,98,6.230134,75,4.767959,47,2.987921,57,3.623649
3,2,0.505051,1,0.252525,0,0,0,0,2,0.505051,...,2,0.505051,0,0,0,0,0,0,0,0
4,1,0.273973,1,0.273973,0,0,0,0,2,0.547945,...,0,0,2,0.547945,0,0,1,0.273973,0,0
5,1,0,2,0,1,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,549,0,580,0,592,0,321,0,282,0,...,396,0,431,0,448,0,440,0,401,0
229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
enfermedad = 'HVB'
rango_de_edad = 'RN'
df_rn_HVB_final = define_multilevel_col(df_rn_HVB,enfermedad, rango_de_edad)
df_rn_HVB_stacked = df_rn_HVB_final.stack(level = [0,1,2])
df_rn_HVB_stacked

% CANTIDAD
0   RN HVB ABRIL        5.639432      127
           AGOSTO       7.015986      158
           DICIEMBRE    4.795737      108
           ENERO      117.272579       64
           FEBRERO      2.841918       64
...                          ...      ...
231 RN HVB MARZO       17.221053      818
           MAYO        12.273684      583
           NOVIEMBRE   17.010526      808
           OCTUBRE     18.378947      873
           SETIEMBRE   17.221053      818

[2784 rows x 2 columns]

In [ ]:
df_rn_HVB_final

RN                                                               \
         HVB                                                                
       ENERO              FEBRERO               MARZO               ABRIL   
    CANTIDAD           % CANTIDAD          % CANTIDAD          % CANTIDAD   
0         64  117.272579       64   2.841918       57   2.531083      127   
1         28    1.780038       22   1.398601       17   1.080737       57   
2          2    0.505051        1   0.252525        0          0        0   
3          1    0.273973        1   0.273973        0          0        0   
4          1           0        2          0        1          0        0   
..       ...         ...      ...        ...      ...        ...      ...   
227      549           0      580          0      592          0      321   
228        0           0        0          0        0          0        0   
229        0           0        0          0        0          0        0   
230        0           0        0          0        0          0        0   
231      815   17.157895      816  17.178947      818  17.221053      658   

                                    ...                                \
                                    ...                                 
                   MAYO             ...   AGOSTO            SETIEMBRE   
             % CANTIDAD          %  ... CANTIDAD          %  CANTIDAD   
0     5.639432      106   4.706927  ...      158   7.015986       155   
1     3.623649       35   2.225048  ...       91   5.785124        98   
2            0        2   0.505051  ...        2   0.505051         0   
3            0        2   0.547945  ...        0          0         2   
4            0        0          0  ...        2          0         0   
..         ...      ...        ...  ...      ...        ...       ...   
227          0      282          0  ...      396          0       431   
228          0        0          0  ...        0          0         0   
229          0        0          0  ...        0          0         0   
230          0        0          0  ...        0          0         0   
231  13.852632      583  12.273684  ...      818  17.221053       818   

                                                                              
                                                                              
                OCTUBRE            NOVIEMBRE            DICIEMBRE             
             % CANTIDAD          %  CANTIDAD          %  CANTIDAD          %  
0     6.882771      137   6.083481       106   4.706927       108   4.795737  
1     6.230134       75   4.767959        47   2.987921        57   3.623649  
2            0        0          0         0          0         0          0  
3     0.547945        0          0         1   0.273973         0          0  
4            0        0          0         0          0         0          0  
..         ...      ...        ...       ...        ...       ...        ...  
227          0      448          0       440          0       401          0  
228          0        0          0         0          0         0          0  
229          0        0          0         0          0         0          0  
230          0        0          0         0          0         0          0  
231  17.221053      873  18.378947       808  17.010526       766  16.126316  

[232 rows x 24 columns]

## < 1 año - APO

In [ ]:
df_m1_apo = df_cobertura.iloc[:, 56:80]
df_m1_apo.columns = df_m1_apo.iloc[0]
df_m1_apo = df_m1_apo[1:]
df_m1_apo

,ENE,%,FEB,%,MAR,%,ABR,%,MAY,%,...,AGO,%,SEPT,%,OCT,%,NOV,%,DIC,%
1,265,7.410515,213,5.956376,162,4.530201,10,0.279642,22,0.615213,...,200,5.592841,238,6.655481,256,7.158837,345,9.647651,238,6.655481
2,122,6.282183,114,5.870237,83,4.273944,0,0,3,0.15448,...,97,4.994851,132,6.797116,126,6.488157,176,9.062822,122,6.282183
3,19,4.75,10,2.5,20,5,0,0,0,0,...,12,3,6,1.5,7,1.75,10,2.5,10,2.5
4,11,5.238095,24,11.428571,17,8.095238,0,0,3,1.428571,...,9,4.285714,15,7.142857,20,9.52381,18,8.571429,14,6.666667
5,29,5.8,33,6.6,15,3,0,0,0,0,...,17,3.4,39,7.8,31,6.2,22,4.4,36,7.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,35,0,26,0,26,0,1,0,13,0,...,26,0,36,0,43,0,48,0,28,0
229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
enfermedad = 'APO'
rango_de_edad = '<1_AÑO'
df_m1_apo_final = define_multilevel_col(df_m1_apo,enfermedad, rango_de_edad)
df_m1_apo_stacked = df_m1_apo_final.stack(level = [0,1,2])
df_m1_apo_final

<1_AÑO                                                            \
         APO                                                             
       ENERO            FEBRERO               MARZO              ABRIL   
    CANTIDAD         % CANTIDAD          % CANTIDAD         % CANTIDAD   
0        265  7.410515      213   5.956376      162  4.530201       10   
1        122  6.282183      114   5.870237       83  4.273944        0   
2         19      4.75       10        2.5       20         5        0   
3         11  5.238095       24  11.428571       17  8.095238        0   
4         29       5.8       33        6.6       15         3        0   
..       ...       ...      ...        ...      ...       ...      ...   
227       35         0       26          0       26         0        1   
228        0         0        0          0        0         0        0   
229        0         0        0          0        0         0        0   
230        0         0        0          0        0         0        0   
231      705  6.464332      603   5.529067      436  3.997799       52   

                                  ...                                         \
                                  ...                                          
                  MAYO            ...   AGOSTO           SETIEMBRE             
            % CANTIDAD         %  ... CANTIDAD         %  CANTIDAD         %   
0    0.279642       22  0.615213  ...      200  5.592841       238  6.655481   
1           0        3   0.15448  ...       97  4.994851       132  6.797116   
2           0        0         0  ...       12         3         6       1.5   
3           0        3  1.428571  ...        9  4.285714        15  7.142857   
4           0        0         0  ...       17       3.4        39       7.8   
..        ...      ...       ...  ...      ...       ...       ...       ...   
227         0       13         0  ...       26         0        36         0   
228         0        0         0  ...        0         0         0         0   
229         0        0         0  ...        0         0         0         0   
230         0        0         0  ...        0         0         0         0   
231  0.476802      119  1.091142  ...      547  5.015588       688  6.308454   

                                                                
                                                                
     OCTUBRE           NOVIEMBRE           DICIEMBRE            
    CANTIDAD         %  CANTIDAD         %  CANTIDAD         %  
0        256  7.158837       345  9.647651       238  6.655481  
1        126  6.488157       176  9.062822       122  6.282183  
2          7      1.75        10       2.5        10       2.5  
3         20   9.52381        18  8.571429        14  6.666667  
4         31       6.2        22       4.4        36       7.2  
..       ...       ...       ...       ...       ...       ...  
227       43         0        48         0        28         0  
228        0         0         0         0         0         0  
229        0         0         0         0         0         0  
230        0         0         0         0         0         0  
231      768  7.041995       945  8.664955       699  6.409316  

[232 rows x 24 columns]

## < 1 año - PENTA

In [ ]:
df_m1_penta = df_cobertura.iloc[:, 82:106]
df_m1_penta.columns = df_m1_penta.iloc[0]
df_m1_penta = df_m1_penta[1:]
df_m1_penta

,ENE,%,FEB,%,MAR,%,ABR,%,MAY,%,...,AGO,%,SEPT,%,OCT,%,NOV,%,DIC,%
1,266,415.923043,233,6.51566,155,4.334452,9,0.251678,23,0.643177,...,215,6.012304,245,6.85123,286,7.997763,358,10.011186,255,7.130872
2,133,6.84861,129,6.642636,80,4.119464,0,0,3,0.15448,...,113,5.818744,141,7.260556,149,7.672503,193,9.938208,140,7.209063
3,27,6.75,28,7,16,4,0,0,0,0,...,24,6,12,3,34,8.5,27,6.75,25,6.25
4,10,4.761905,21,10,17,8.095238,0,0,3,1.428571,...,9,4.285714,16,7.619048,19,9.047619,13,6.190476,18,8.571429
5,30,6,33,6.6,16,3.2,0,0,0,0,...,19,3.8,42,8.4,30,6,24,4.8,37,7.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,35,0,26,0,27,0,1,0,13,0,...,27,0,36,0,43,0,51,0,30,0
229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
enfermedad = 'PENTA'
rango_de_edad = '<1_AÑO'
df_m1_penta_final = define_multilevel_col(df_m1_penta,enfermedad, rango_de_edad)
df_m1_penta_stacked = df_m1_penta_final.stack(level = [0,1,2])
df_m1_penta_final

<1_AÑO                                                             \
       PENTA                                                              
       ENERO              FEBRERO              MARZO              ABRIL   
    CANTIDAD           % CANTIDAD         % CANTIDAD         % CANTIDAD   
0        266  415.923043      233   6.51566      155  4.334452        9   
1        133     6.84861      129  6.642636       80  4.119464        0   
2         27        6.75       28         7       16         4        0   
3         10    4.761905       21        10       17  8.095238        0   
4         30           6       33       6.6       16       3.2        0   
..       ...         ...      ...       ...      ...       ...      ...   
227       35           0       26         0       27         0        1   
228        0           0        0         0        0         0        0   
229        0           0        0         0        0         0        0   
230        0           0        0         0        0         0        0   
231      695    6.372639      629  5.767467      438  4.016138       54   

                                  ...                                         \
                                  ...                                          
                  MAYO            ...   AGOSTO           SETIEMBRE             
            % CANTIDAD         %  ... CANTIDAD         %  CANTIDAD         %   
0    0.251678       23  0.643177  ...      215  6.012304       245   6.85123   
1           0        3   0.15448  ...      113  5.818744       141  7.260556   
2           0        0         0  ...       24         6        12         3   
3           0        3  1.428571  ...        9  4.285714        16  7.619048   
4           0        0         0  ...       19       3.8        42       8.4   
..        ...      ...       ...  ...      ...       ...       ...       ...   
227         0       13         0  ...       27         0        36         0   
228         0        0         0  ...        0         0         0         0   
229         0        0         0  ...        0         0         0         0   
230         0        0         0  ...        0         0         0         0   
231   0.49514      122   1.11865  ...      559  5.125619       694   6.36347   

                                                                 
                                                                 
     OCTUBRE           NOVIEMBRE            DICIEMBRE            
    CANTIDAD         %  CANTIDAD          %  CANTIDAD         %  
0        286  7.997763       358  10.011186       255  7.130872  
1        149  7.672503       193   9.938208       140  7.209063  
2         34       8.5        27       6.75        25      6.25  
3         19  9.047619        13   6.190476        18  8.571429  
4         30         6        24        4.8        37       7.4  
..       ...       ...       ...        ...       ...       ...  
227       43         0        51          0        30         0  
228        0         0         0          0         0         0  
229        0         0         0          0         0         0  
230        0         0         0          0         0         0  
231      790  7.243719       949   8.701632       731  6.702732  

[232 rows x 24 columns]

## < 1 año - ROTA

In [ ]:
df_m1_rota = df_cobertura.iloc[:, 108:132]
df_m1_rota.columns = df_m1_rota.iloc[0]
df_m1_rota = df_m1_rota[1:]
df_m1_rota

,ENE,%,FEB,%,MAR,%,ABR,%,MAY,%,...,AGO,%,SEPT,%,OCT,%,NOV,%,DIC,%
1,294,442.1127,231,6.459732,170,4.753915,15,0.419463,39,1.090604,...,315,8.808725,401,11.213647,287,8.025727,257,7.186801,253,7.074944
2,140,7.209063,127,6.53965,79,4.067971,0,0,8,0.411946,...,190,9.783728,233,11.99794,160,8.238929,141,7.260556,129,6.642636
3,12,3,15,3.75,2,0.5,0,0,0,0,...,16,4,11,2.75,9,2.25,4,1,6,1.5
4,31,14.761905,19,9.047619,9,4.285714,0,0,8,3.809524,...,22,10.47619,30,14.285714,21,10,10,4.761905,10,4.761905
5,27,5.4,28,5.6,18,3.6,0,0,0,0,...,31,6.2,45,9,38,7.6,27,5.4,45,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,41,0,30,0,42,0,5,0,16,0,...,49,0,46,0,25,0,27,0,31,0
229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
enfermedad = 'ROTA'
rango_de_edad = '<1_AÑO'
df_m1_rota_final = define_multilevel_col(df_m1_rota,enfermedad, rango_de_edad)
df_m1_rota_stacked = df_m1_rota_final.stack(level = [0,1,2])
df_m1_rota_final

<1_AÑO                                                            \
        ROTA                                                             
       ENERO             FEBRERO              MARZO              ABRIL   
    CANTIDAD          % CANTIDAD         % CANTIDAD         % CANTIDAD   
0        294   442.1127      231  6.459732      170  4.753915       15   
1        140   7.209063      127   6.53965       79  4.067971        0   
2         12          3       15      3.75        2       0.5        0   
3         31  14.761905       19  9.047619        9  4.285714        0   
4         27        5.4       28       5.6       18       3.6        0   
..       ...        ...      ...       ...      ...       ...      ...   
227       41          0       30         0       42         0        5   
228        0          0        0         0        0         0        0   
229        0          0        0         0        0         0        0   
230        0          0        0         0        0         0        0   
231      789    7.23455      673  6.170915      466  4.272877       76   

                                  ...                                          \
                                  ...                                           
                  MAYO            ...   AGOSTO           SETIEMBRE              
            % CANTIDAD         %  ... CANTIDAD         %  CANTIDAD          %   
0    0.419463       39  1.090604  ...      315  8.808725       401  11.213647   
1           0        8  0.411946  ...      190  9.783728       233   11.99794   
2           0        0         0  ...       16         4        11       2.75   
3           0        8  3.809524  ...       22  10.47619        30  14.285714   
4           0        0         0  ...       31       6.2        45          9   
..        ...      ...       ...  ...      ...       ...       ...        ...   
227         0       16         0  ...       49         0        46          0   
228         0        0         0  ...        0         0         0          0   
229         0        0         0  ...        0         0         0          0   
230         0        0         0  ...        0         0         0          0   
231  0.696864      153  1.402897  ...      793  7.271227       990   9.077572   

                                                                
                                                                
     OCTUBRE           NOVIEMBRE           DICIEMBRE            
    CANTIDAD         %  CANTIDAD         %  CANTIDAD         %  
0        287  8.025727       257  7.186801       253  7.074944  
1        160  8.238929       141  7.260556       129  6.642636  
2          9      2.25         4         1         6       1.5  
3         21        10        10  4.761905        10  4.761905  
4         38       7.6        27       5.4        45         9  
..       ...       ...       ...       ...       ...       ...  
227       25         0        27         0        31         0  
228        0         0         0         0         0         0  
229        0         0         0         0         0         0  
230        0         0         0         0         0         0  
231      791  7.252888       749  6.867779       709  6.501009  

[232 rows x 24 columns]

## 1 año - NEUMO


In [ ]:
def preprocessing_enf_edad(df, col):
  df_final = df.iloc[:, col:col+24]
  df_final.columns = df_final.iloc[0]
  return df_final[1:]

In [ ]:
df_m1_neumo = preprocessing_enf_edad(df_cobertura, 135)
df_m1_neumo

,ENE,%,FEB,%,MAR,%,ABR,%,MAY,%,...,AGO,%,SEPT,%,OCT,%,NOV,%,DIC,%
1,208,4.797048,214,4.935424,143,3.29797,15,0.345941,33,0.76107,...,222,5.119926,386,8.902214,300,6.918819,318,7.333948,206,4.750923
2,88,3.775204,96,4.118404,68,2.917203,0,0,2,0.0858,...,110,4.719005,245,10.510511,149,6.392106,160,6.864007,109,4.676105
3,0,0,14,3.535354,11,2.777778,0,0,0,0,...,13,3.282828,27,6.818182,15,3.787879,20,5.050505,10,2.525253
4,13,3.561644,11,3.013699,7,1.917808,0,0,2,0.547945,...,12,3.287671,13,3.561644,39,10.684932,4,1.09589,13,3.561644
5,26,4.642857,31,5.535714,19,3.392857,0,0,0,0,...,19,3.392857,72,12.857143,21,3.75,33,5.892857,45,8.035714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,41,0,28,0,18,0,0,0,9,0,...,46,0,48,0,33,0,31,0,22,0
229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
enfermedad = 'NEUMO'
rango_de_edad = '1_AÑO'
df_m1_neumo_final = define_multilevel_col(df_m1_neumo,enfermedad, rango_de_edad)
df_m1_neumo_stacked = df_m1_neumo_final.stack(level = [0,1,2])
df_m1_neumo_final

1_AÑO                                                           \
       NEUMO                                                            
       ENERO            FEBRERO              MARZO              ABRIL   
    CANTIDAD         % CANTIDAD         % CANTIDAD         % CANTIDAD   
0        208  4.797048      214  4.935424      143   3.29797       15   
1         88  3.775204       96  4.118404       68  2.917203        0   
2          0         0       14  3.535354       11  2.777778        0   
3         13  3.561644       11  3.013699        7  1.917808        0   
4         26  4.642857       31  5.535714       19  3.392857        0   
..       ...       ...      ...       ...      ...       ...      ...   
227       41         0       28         0       18         0        0   
228        0         0        0         0        0         0        0   
229        0         0        0         0        0         0        0   
230        0         0        0         0        0         0        0   
231      598  4.746408      606  4.809906      370  2.936741       69   

                                  ...                                          \
                                  ...                                           
                  MAYO            ...   AGOSTO           SETIEMBRE              
            % CANTIDAD         %  ... CANTIDAD         %  CANTIDAD          %   
0    0.345941       33   0.76107  ...      222  5.119926       386   8.902214   
1           0        2    0.0858  ...      110  4.719005       245  10.510511   
2           0        0         0  ...       13  3.282828        27   6.818182   
3           0        2  0.547945  ...       12  3.287671        13   3.561644   
4           0        0         0  ...       19  3.392857        72  12.857143   
..        ...      ...       ...  ...      ...       ...       ...        ...   
227         0        9         0  ...       46         0        48          0   
228         0        0         0  ...        0         0         0          0   
229         0        0         0  ...        0         0         0          0   
230         0        0         0  ...        0         0         0          0   
231  0.547663      110  0.873085  ...      686  5.444877       977   7.754584   

                                                                 
                                                                 
     OCTUBRE            NOVIEMBRE           DICIEMBRE            
    CANTIDAD          %  CANTIDAD         %  CANTIDAD         %  
0        300   6.918819       318  7.333948       206  4.750923  
1        149   6.392106       160  6.864007       109  4.676105  
2         15   3.787879        20  5.050505        10  2.525253  
3         39  10.684932         4   1.09589        13  3.561644  
4         21       3.75        33  5.892857        45  8.035714  
..       ...        ...       ...       ...       ...       ...  
227       33          0        31         0        22         0  
228        0          0         0         0         0         0  
229        0          0         0         0         0         0  
230        0          0         0         0         0         0  
231      960   7.619652       878  6.968807       586  4.651163  

[232 rows x 24 columns]

## 1 año - SPR


In [ ]:
df_m1_spr = preprocessing_enf_edad(df_cobertura, 161)
df_m1_spr

,ENE,%,FEB,%,MAR,%,ABR,%,MAY,%,...,AGO,%,SEPT,%,OCT,%,NOV,%,DIC,%
1,156,3.597786,143,3.29797,99,2.28321,7,0.161439,11,0.25369,...,164,3.782288,249,5.74262,116,2.675277,139,3.20572,114,2.629151
2,66,2.831403,58,2.488202,44,1.887602,0,0,0,0,...,73,3.131703,138,5.920206,49,2.102102,66,2.831403,50,2.145002
3,10,2.525253,9,2.272727,7,1.767677,0,0,0,0,...,17,4.292929,14,3.535354,10,2.525253,11,2.777778,9,2.272727
4,4,1.09589,12,3.287671,5,1.369863,0,0,0,0,...,12,3.287671,9,2.465753,12,3.287671,4,1.09589,2,0.547945
5,14,2.5,8,1.428571,7,1.25,0,0,0,0,...,8,1.428571,38,6.785714,7,1.25,12,2.142857,21,3.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,19,0,16,0,18,0,1,0,8,0,...,23,0,28,0,23,0,35,0,20,0
229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
enfermedad = 'SPR'
rango_de_edad = '1_AÑO'
df_m1_spr_final = define_multilevel_col(df_m1_spr,enfermedad, rango_de_edad)
df_m1_spr_stacked = df_m1_spr_final.stack(level = [0,1,2])
df_m1_spr_final

1_AÑO                                                           \
         SPR                                                            
       ENERO            FEBRERO              MARZO              ABRIL   
    CANTIDAD         % CANTIDAD         % CANTIDAD         % CANTIDAD   
0        156  3.597786      143   3.29797       99   2.28321        7   
1         66  2.831403       58  2.488202       44  1.887602        0   
2         10  2.525253        9  2.272727        7  1.767677        0   
3          4   1.09589       12  3.287671        5  1.369863        0   
4         14       2.5        8  1.428571        7      1.25        0   
..       ...       ...      ...       ...      ...       ...      ...   
227       19         0       16         0       18         0        1   
228        0         0        0         0        0         0        0   
229        0         0        0         0        0         0        0   
230        0         0        0         0        0         0        0   
231      384  3.047861      379  3.008175      259  2.055719       33   

                                  ...                                         \
                                  ...                                          
                  MAYO            ...   AGOSTO           SETIEMBRE             
            % CANTIDAD         %  ... CANTIDAD         %  CANTIDAD         %   
0    0.161439       11   0.25369  ...      164  3.782288       249   5.74262   
1           0        0         0  ...       73  3.131703       138  5.920206   
2           0        0         0  ...       17  4.292929        14  3.535354   
3           0        0         0  ...       12  3.287671         9  2.465753   
4           0        0         0  ...        8  1.428571        38  6.785714   
..        ...      ...       ...  ...      ...       ...       ...       ...   
227         0        8         0  ...       23         0        28         0   
228         0        0         0  ...        0         0         0         0   
229         0        0         0  ...        0         0         0         0   
230         0        0         0  ...        0         0         0         0   
231  0.261926       62  0.492103  ...      419  3.325661       596  4.730534   

                                                                
                                                                
     OCTUBRE           NOVIEMBRE           DICIEMBRE            
    CANTIDAD         %  CANTIDAD         %  CANTIDAD         %  
0        116  2.675277       139   3.20572       114  2.629151  
1         49  2.102102        66  2.831403        50  2.145002  
2         10  2.525253        11  2.777778         9  2.272727  
3         12  3.287671         4   1.09589         2  0.547945  
4          7      1.25        12  2.142857        21      3.75  
..       ...       ...       ...       ...       ...       ...  
227       23         0        35         0        20         0  
228        0         0         0         0         0         0  
229        0         0         0         0         0         0  
230        0         0         0         0         0         0  
231      446  3.539963       396  3.143107       279  2.214461  

[232 rows x 24 columns]

## 1 año - DPT


In [ ]:
df_m1_dpt = preprocessing_enf_edad(df_cobertura, 187)
df_m1_dpt

,ENE,%,FEB,%,MAR,%,ABR,%,MAY,%,...,AGO,%,SEPT,%,OCT,%,NOV,%,DIC,%
1,155,3.574723,145,3.344096,98,2.260148,8,0.184502,12,0.276753,...,150,3.45941,187,4.312731,156,3.597786,192,4.428044,136,3.136531
2,80,3.432003,68,2.917203,47,2.016302,0,0,0,0,...,66,2.831403,76,3.260403,68,2.917203,95,4.075504,55,2.359502
3,10,2.525253,12,3.030303,4,1.010101,0,0,0,0,...,13,3.282828,5,1.262626,4,1.010101,7,1.767677,7,1.767677
4,4,1.09589,12,3.287671,7,1.917808,0,0,0,0,...,9,2.465753,9,2.465753,18,4.931507,10,2.739726,4,1.09589
5,19,3.392857,9,1.607143,9,1.607143,0,0,0,0,...,10,1.785714,14,2.5,9,1.607143,18,3.214286,8,1.428571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,21,0,16,0,18,0,1,0,8,0,...,26,0,23,0,26,0,44,0,27,0
229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
enfermedad = 'APT'
rango_de_edad = '1_AÑO'
df_m1_dpt_final = define_multilevel_col(df_m1_dpt,enfermedad, rango_de_edad)
df_m1_dpt_stacked = df_m1_dpt_final.stack(level = [0,1,2])
df_m1_dpt_final

1_AÑO                                                           \
         APT                                                            
       ENERO            FEBRERO              MARZO              ABRIL   
    CANTIDAD         % CANTIDAD         % CANTIDAD         % CANTIDAD   
0        155  3.574723      145  3.344096       98  2.260148        8   
1         80  3.432003       68  2.917203       47  2.016302        0   
2         10  2.525253       12  3.030303        4  1.010101        0   
3          4   1.09589       12  3.287671        7  1.917808        0   
4         19  3.392857        9  1.607143        9  1.607143        0   
..       ...       ...      ...       ...      ...       ...      ...   
227       21         0       16         0       18         0        1   
228        0         0        0         0        0         0        0   
229        0         0        0         0        0         0        0   
230        0         0        0         0        0         0        0   
231      403  3.198667      387  3.071672      272  2.158901       38   

                                  ...                                         \
                                  ...                                          
                  MAYO            ...   AGOSTO           SETIEMBRE             
            % CANTIDAD         %  ... CANTIDAD         %  CANTIDAD         %   
0    0.184502       12  0.276753  ...      150   3.45941       187  4.312731   
1           0        0         0  ...       66  2.831403        76  3.260403   
2           0        0         0  ...       13  3.282828         5  1.262626   
3           0        0         0  ...        9  2.465753         9  2.465753   
4           0        0         0  ...       10  1.785714        14       2.5   
..        ...      ...       ...  ...      ...       ...       ...       ...   
227         0        8         0  ...       26         0        23         0   
228         0        0         0  ...        0         0         0         0   
229         0        0         0  ...        0         0         0         0   
230         0        0         0  ...        0         0         0         0   
231  0.301611       63   0.50004  ...      427  3.389158       568  4.508294   

                                                                
                                                                
     OCTUBRE           NOVIEMBRE           DICIEMBRE            
    CANTIDAD         %  CANTIDAD         %  CANTIDAD         %  
0        156  3.597786       192  4.428044       136  3.136531  
1         68  2.917203        95  4.075504        55  2.359502  
2          4  1.010101         7  1.767677         7  1.767677  
3         18  4.931507        10  2.739726         4   1.09589  
4          9  1.607143        18  3.214286         8  1.428571  
..       ...       ...       ...       ...       ...       ...  
227       26         0        44         0        27         0  
228        0         0         0         0         0         0  
229        0         0         0         0         0         0  
230        0         0         0         0         0         0  
231      589  4.674974       645  5.119454       398  3.158981  

[232 rows x 24 columns]

## 1 año - APO_REF


In [ ]:
df_m1_aporef = preprocessing_enf_edad(df_cobertura, 213)
df_m1_aporef

,ENE,%,FEB,%,MAR,%,ABR,%,MAY,%,...,AGO,%,SEPT,%,OCT,%,NOV,%,DIC,%
1,142,3.274908,128,2.95203,94,2.167897,8,0.184502,9,0.207565,...,140,3.228782,263,6.065498,180,4.151292,241,5.558118,160,3.690037
2,65,2.788503,53,2.273702,47,2.016302,0,0,0,0,...,58,2.488202,142,6.091806,85,3.646504,128,5.491205,77,3.303303
3,2,0.505051,1,0.252525,4,1.010101,0,0,0,0,...,2,0.505051,3,0.757576,4,1.010101,15,3.787879,8,2.020202
4,5,1.369863,13,3.561644,7,1.917808,0,0,0,0,...,13,3.561644,8,2.191781,24,6.575342,10,2.739726,4,1.09589
5,16,2.857143,9,1.607143,12,2.142857,0,0,0,0,...,10,1.785714,52,9.285714,14,2.5,26,4.642857,20,3.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,19,0,15,0,20,0,1,0,7,0,...,25,0,25,0,28,0,42,0,26,0
229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
enfermedad = 'APO_REF'
rango_de_edad = '1_AÑO'
df_m1_aporef_final = define_multilevel_col(df_m1_aporef,enfermedad, rango_de_edad)
df_m1_aporef_stacked = df_m1_aporef_final.stack(level = [0,1,2])
df_m1_aporef_final

1_AÑO                                                           \
     APO_REF                                                            
       ENERO            FEBRERO              MARZO              ABRIL   
    CANTIDAD         % CANTIDAD         % CANTIDAD         % CANTIDAD   
0        142  3.274908      128   2.95203       94  2.167897        8   
1         65  2.788503       53  2.273702       47  2.016302        0   
2          2  0.505051        1  0.252525        4  1.010101        0   
3          5  1.369863       13  3.561644        7  1.917808        0   
4         16  2.857143        9  1.607143       12  2.142857        0   
..       ...       ...      ...       ...      ...       ...      ...   
227       19         0       15         0       20         0        1   
228        0         0        0         0        0         0        0   
229        0         0        0         0        0         0        0   
230        0         0        0         0        0         0        0   
231      382  3.031987      352  2.793873      267  2.119216       32   

                                  ...                                         \
                                  ...                                          
                  MAYO            ...   AGOSTO           SETIEMBRE             
            % CANTIDAD         %  ... CANTIDAD         %  CANTIDAD         %   
0    0.184502        9  0.207565  ...      140  3.228782       263  6.065498   
1           0        0         0  ...       58  2.488202       142  6.091806   
2           0        0         0  ...        2  0.505051         3  0.757576   
3           0        0         0  ...       13  3.561644         8  2.191781   
4           0        0         0  ...       10  1.785714        52  9.285714   
..        ...      ...       ...  ...      ...       ...       ...       ...   
227         0        7         0  ...       25         0        25         0   
228         0        0         0  ...        0         0         0         0   
229         0        0         0  ...        0         0         0         0   
230         0        0         0  ...        0         0         0         0   
231  0.253988       57  0.452417  ...      415  3.293912       670  5.317882   

                                                                
                                                                
     OCTUBRE           NOVIEMBRE           DICIEMBRE            
    CANTIDAD         %  CANTIDAD         %  CANTIDAD         %  
0        180  4.151292       241  5.558118       160  3.690037  
1         85  3.646504       128  5.491205        77  3.303303  
2          4  1.010101        15  3.787879         8  2.020202  
3         24  6.575342        10  2.739726         4   1.09589  
4         14       2.5        26  4.642857        20  3.571429  
..       ...       ...       ...       ...       ...       ...  
227       28         0        42         0        26         0  
228        0         0         0         0         0         0  
229        0         0         0         0         0         0  
230        0         0         0         0         0         0  
231      614  4.873403       683  5.421065       427  3.389158  

[232 rows x 24 columns]

## 1 año - AMA


In [ ]:
df_m1_ama = preprocessing_enf_edad(df_cobertura, 239)
df_m1_ama

,ENE,%,FEB,%,MAR,%,ABR,%,MAY,%,...,AGO,%,SEPT,%,OCT,%,NOV,%,DIC,%
1,150,3.45941,141,3.251845,83,1.914207,2,0.046125,8,0.184502,...,180,4.151292,207,4.773985,220,5.073801,247,5.696494,181,4.174354
2,71,3.045903,55,2.359502,26,1.115401,0,0,0,0,...,77,3.303303,109,4.676105,92,3.946804,125,5.362505,91,3.903904
3,10,2.525253,13,3.282828,9,2.272727,0,0,0,0,...,17,4.292929,19,4.79798,22,5.555556,16,4.040404,4,1.010101
4,9,2.465753,9,2.465753,0,0,0,0,0,0,...,11,3.013699,3,0.821918,10,2.739726,11,3.013699,4,1.09589
5,20,3.571429,2,0.357143,0,0,0,0,0,0,...,9,1.607143,9,1.607143,12,2.142857,10,1.785714,55,9.821429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,14,0,29,0,3,0,0,0,5,0,...,48,0,45,0,28,0,47,0,46,0
229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
enfermedad = 'AMA'
rango_de_edad = '1_AÑO'
df_m1_ama_final = define_multilevel_col(df_m1_ama,enfermedad, rango_de_edad)
df_m1_ama_stacked = df_m1_ama_final.stack(level = [0,1,2])
df_m1_ama_final

1_AÑO                                                           \
         AMA                                                            
       ENERO            FEBRERO              MARZO              ABRIL   
    CANTIDAD         % CANTIDAD         % CANTIDAD         % CANTIDAD   
0        150   3.45941      141  3.251845       83  1.914207        2   
1         71  3.045903       55  2.359502       26  1.115401        0   
2         10  2.525253       13  3.282828        9  2.272727        0   
3          9  2.465753        9  2.465753        0         0        0   
4         20  3.571429        2  0.357143        0         0        0   
..       ...       ...      ...       ...      ...       ...      ...   
227       14         0       29         0        3         0        0   
228        0         0        0         0        0         0        0   
229        0         0        0         0        0         0        0   
230        0         0        0         0        0         0        0   
231      349  2.770061      344  2.730375      202  1.603302       17   

                                  ...                                         \
                                  ...                                          
                  MAYO            ...   AGOSTO           SETIEMBRE             
            % CANTIDAD         %  ... CANTIDAD         %  CANTIDAD         %   
0    0.046125        8  0.184502  ...      180  4.151292       207  4.773985   
1           0        0         0  ...       77  3.303303       109  4.676105   
2           0        0         0  ...       17  4.292929        19   4.79798   
3           0        0         0  ...       11  3.013699         3  0.821918   
4           0        0         0  ...        9  1.607143         9  1.607143   
..        ...      ...       ...  ...      ...       ...       ...       ...   
227         0        5         0  ...       48         0        45         0   
228         0        0         0  ...        0         0         0         0   
229         0        0         0  ...        0         0         0         0   
230         0        0         0  ...        0         0         0         0   
231  0.134931       69  0.547663  ...      470  3.730455       554  4.397174   

                                                                
                                                                
     OCTUBRE           NOVIEMBRE           DICIEMBRE            
    CANTIDAD         %  CANTIDAD         %  CANTIDAD         %  
0        220  5.073801       247  5.696494       181  4.174354  
1         92  3.946804       125  5.362505        91  3.903904  
2         22  5.555556        16  4.040404         4  1.010101  
3         10  2.739726        11  3.013699         4   1.09589  
4         12  2.142857        10  1.785714        55  9.821429  
..       ...       ...       ...       ...       ...       ...  
227       28         0        47         0        46         0  
228        0         0         0         0         0         0  
229        0         0         0         0         0         0  
230        0         0         0         0         0         0  
231      735  5.833796       759  6.024288       561  4.452734  

[232 rows x 24 columns]

## 1 año - VARI


In [ ]:
df_m1_vari = preprocessing_enf_edad(df_cobertura, 265)
df_m1_vari

,ENE,%,FEB,%,MAR,%,ABR,%,MAY,%,...,AGO,%,SEPT,%,OCT,%,NOV,%,DIC,%
1,187,4.312731,184,4.243542,126,2.905904,15,0.345941,23,0.530443,...,203,4.681734,323,7.449262,251,5.788745,265,6.111624,138,3.182657
2,81,3.474903,87,3.732304,61,2.616903,0,0,1,0.0429,...,108,4.633205,185,7.936508,119,5.105105,138,5.920206,68,2.917203
3,27,6.818182,17,4.292929,15,3.787879,0,0,0,0,...,15,3.787879,25,6.313131,14,3.535354,37,9.343434,10,2.525253
4,3,0.821918,14,3.835616,4,1.09589,0,0,1,0.273973,...,4,1.09589,4,1.09589,26,7.123288,0,0,0,0
5,17,3.035714,28,5,14,2.5,0,0,0,0,...,21,3.75,66,11.785714,22,3.928571,27,4.821429,31,5.535714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,36,0,31,0,21,0,0,0,10,0,...,37,0,35,0,39,0,35,0,27,0
229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
enfermedad = 'VARI'
rango_de_edad = '1_AÑO'
df_m1_vari_final = define_multilevel_col(df_m1_vari,enfermedad, rango_de_edad)
df_m1_vari_stacked = df_m1_vari_final.stack(level = [0,1,2])
df_m1_vari_final

1_AÑO                                                           \
        VARI                                                            
       ENERO            FEBRERO              MARZO              ABRIL   
    CANTIDAD         % CANTIDAD         % CANTIDAD         % CANTIDAD   
0        187  4.312731      184  4.243542      126  2.905904       15   
1         81  3.474903       87  3.732304       61  2.616903        0   
2         27  6.818182       17  4.292929       15  3.787879        0   
3          3  0.821918       14  3.835616        4   1.09589        0   
4         17  3.035714       28         5       14       2.5        0   
..       ...       ...      ...       ...      ...       ...      ...   
227       36         0       31         0       21         0        0   
228        0         0        0         0        0         0        0   
229        0         0        0         0        0         0        0   
230        0         0        0         0        0         0        0   
231      448  3.555838      423  3.357409      305  2.420827       69   

                                  ...                                          \
                                  ...                                           
                  MAYO            ...   AGOSTO           SETIEMBRE              
            % CANTIDAD         %  ... CANTIDAD         %  CANTIDAD          %   
0    0.345941       23  0.530443  ...      203  4.681734       323   7.449262   
1           0        1    0.0429  ...      108  4.633205       185   7.936508   
2           0        0         0  ...       15  3.787879        25   6.313131   
3           0        1  0.273973  ...        4   1.09589         4    1.09589   
4           0        0         0  ...       21      3.75        66  11.785714   
..        ...      ...       ...  ...      ...       ...       ...        ...   
227         0       10         0  ...       37         0        35          0   
228         0        0         0  ...        0         0         0          0   
229         0        0         0  ...        0         0         0          0   
230         0        0         0  ...        0         0         0          0   
231  0.547663       83  0.658782  ...      553  4.389237       753   5.976665   

                                                                
                                                                
     OCTUBRE           NOVIEMBRE           DICIEMBRE            
    CANTIDAD         %  CANTIDAD         %  CANTIDAD         %  
0        251  5.788745       265  6.111624       138  3.182657  
1        119  5.105105       138  5.920206        68  2.917203  
2         14  3.535354        37  9.343434        10  2.525253  
3         26  7.123288         0         0         0         0  
4         22  3.928571        27  4.821429        31  5.535714  
..       ...       ...       ...       ...       ...       ...  
227       39         0        35         0        27         0  
228        0         0         0         0         0         0  
229        0         0         0         0         0         0  
230        0         0         0         0         0         0  
231      820  6.508453       670  5.317882       473  3.754266  

[232 rows x 24 columns]

## 2 año - influenza

In [ ]:
df_2_infllu = preprocessing_enf_edad(df_cobertura, 292)
df_2_infllu

,ENE,%,FEB,%,MAR,%,ABR,%,MAY,%,...,AGO,%,SEPT,%,OCT,%,NOV,%,DIC,%
1,100,2.424242,62,1.50303,46,1.115152,2,0.048485,1,0.024242,...,136,3.29697,418,10.133333,252,6.109091,288,6.981818,158,3.830303
2,33,1.486486,11,0.495495,22,0.990991,0,0,0,0,...,47,2.117117,263,11.846847,134,6.036036,148,6.666667,104,4.684685
3,10,2.597403,4,1.038961,0,0,0,0,0,0,...,10,2.597403,22,5.714286,18,4.675325,37,9.61039,16,4.155844
4,2,0.484262,2,0.484262,2,0.484262,0,0,0,0,...,4,0.968523,12,2.905569,63,15.254237,11,2.663438,8,1.937046
5,0,0,0,0,6,1.2,0,0,0,0,...,0,0,88,17.6,11,2.2,12,2.4,36,7.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,3,0,11,0,7,0,0,0,3,0,...,8,0,18,0,14,0,19,0,11,0
229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
enfermedad = 'INFLUENZA'
rango_de_edad = '2_AÑOS'
df_2_infllu_final = define_multilevel_col(df_2_infllu,enfermedad, rango_de_edad)
df_2_infllu_stacked = df_2_infllu_final.stack(level = [0,1,2])
df_2_infllu_final

2_AÑOS                                                           \
    INFLUENZA                                                            
        ENERO            FEBRERO              MARZO              ABRIL   
     CANTIDAD         % CANTIDAD         % CANTIDAD         % CANTIDAD   
0         100  2.424242       62   1.50303       46  1.115152        2   
1          33  1.486486       11  0.495495       22  0.990991        0   
2          10  2.597403        4  1.038961        0         0        0   
3           2  0.484262        2  0.484262        2  0.484262        0   
4           0         0        0         0        6       1.2        0   
..        ...       ...      ...       ...      ...       ...      ...   
227         3         0       11         0        7         0        0   
228         0         0        0         0        0         0        0   
229         0         0        0         0        0         0        0   
230         0         0        0         0        0         0        0   
231       226   2.09298      161  1.491017      175   1.62067        8   

                                  ...                                          \
                                  ...                                           
                  MAYO            ...   AGOSTO           SETIEMBRE              
            % CANTIDAD         %  ... CANTIDAD         %  CANTIDAD          %   
0    0.048485        1  0.024242  ...      136   3.29697       418  10.133333   
1           0        0         0  ...       47  2.117117       263  11.846847   
2           0        0         0  ...       10  2.597403        22   5.714286   
3           0        0         0  ...        4  0.968523        12   2.905569   
4           0        0         0  ...        0         0        88       17.6   
..        ...      ...       ...  ...      ...       ...       ...        ...   
227         0        3         0  ...        8         0        18          0   
228         0        0         0  ...        0         0         0          0   
229         0        0         0  ...        0         0         0          0   
230         0        0         0  ...        0         0         0          0   
231  0.074088       41    0.3797  ...      406  3.759956       892   8.260789   

                                                                 
                                                                 
     OCTUBRE            NOVIEMBRE           DICIEMBRE            
    CANTIDAD          %  CANTIDAD         %  CANTIDAD         %  
0        252   6.109091       288  6.981818       158  3.830303  
1        134   6.036036       148  6.666667       104  4.684685  
2         18   4.675325        37   9.61039        16  4.155844  
3         63  15.254237        11  2.663438         8  1.937046  
4         11        2.2        12       2.4        36       7.2  
..       ...        ...       ...       ...       ...       ...  
227       14          0        19         0        11         0  
228        0          0         0         0         0         0  
229        0          0         0         0         0         0  
230        0          0         0         0         0         0  
231      816   7.556955       763  7.066123       346  3.204297  

[232 rows x 24 columns]

## 3 años - influenza

In [ ]:
df_3_infllu = preprocessing_enf_edad(df_cobertura, 292+27)
df_3_infllu

,ENE,%,FEB,%,MAR,%,ABR,%,MAY,%,...,AGO,%,SEPT,%,OCT,%,NOV,%,DIC,%
1,60,10.752688,58,10.394265,43,7.706093,1,0.179211,3,0.537634,...,104,18.637993,390,69.892473,304,54.480287,327,58.602151,128,22.939068
2,20,6.578947,15,4.934211,25,8.223684,0,0,1,0.328947,...,33,10.855263,239,78.618421,158,51.973684,154,50.657895,71,23.355263
3,6,12.765957,4,8.510638,1,2.12766,0,0,0,0,...,10,21.276596,18,38.297872,12,25.531915,38,80.851064,6,12.765957
4,2,4.878049,1,2.439024,0,0,0,0,1,2.439024,...,2,4.878049,0,0,90,219.512195,17,41.463415,4,9.756098
5,1,1.449275,2,2.898551,9,13.043478,0,0,0,0,...,0,0,67,97.101449,10,14.492754,12,17.391304,27,39.130435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,8,0,11,0,7,0,0,0,2,0,...,9,0,3,0,9,0,20,0,6,0
229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
enfermedad = 'INFLUENZA'
rango_de_edad = '3_AÑOS'
df_3_infllu_final = define_multilevel_col(df_3_infllu,enfermedad, rango_de_edad)
df_3_infllu_stacked = df_3_infllu_final.stack(level = [0,1,2])
df_3_infllu_final

3_AÑOS                                                              \
    INFLUENZA                                                               
        ENERO             FEBRERO               MARZO               ABRIL   
     CANTIDAD          % CANTIDAD          % CANTIDAD          % CANTIDAD   
0          60  10.752688       58  10.394265       43   7.706093        1   
1          20   6.578947       15   4.934211       25   8.223684        0   
2           6  12.765957        4   8.510638        1    2.12766        0   
3           2   4.878049        1   2.439024        0          0        0   
4           1   1.449275        2   2.898551        9  13.043478        0   
..        ...        ...      ...        ...      ...        ...      ...   
227         8          0       11          0        7          0        0   
228         0          0        0          0        0          0        0   
229         0          0        0          0        0          0        0   
230         0          0        0          0        0          0        0   
231       173  11.649832      148    9.96633      155   10.43771        8   

                                  ...                                \
                                  ...                                 
                  MAYO            ...   AGOSTO            SETIEMBRE   
            % CANTIDAD         %  ... CANTIDAD          %  CANTIDAD   
0    0.179211        3  0.537634  ...      104  18.637993       390   
1           0        1  0.328947  ...       33  10.855263       239   
2           0        0         0  ...       10  21.276596        18   
3           0        1  2.439024  ...        2   4.878049         0   
4           0        0         0  ...        0          0        67   
..        ...      ...       ...  ...      ...        ...       ...   
227         0        2         0  ...        9          0         3   
228         0        0         0  ...        0          0         0   
229         0        0         0  ...        0          0         0   
230         0        0         0  ...        0          0         0   
231  0.538721       26  1.750842  ...      317  21.346801       854   

                                                                               
                                                                               
                OCTUBRE             NOVIEMBRE            DICIEMBRE             
             % CANTIDAD           %  CANTIDAD          %  CANTIDAD          %  
0    69.892473      304   54.480287       327  58.602151       128  22.939068  
1    78.618421      158   51.973684       154  50.657895        71  23.355263  
2    38.297872       12   25.531915        38  80.851064         6  12.765957  
3            0       90  219.512195        17  41.463415         4   9.756098  
4    97.101449       10   14.492754        12  17.391304        27  39.130435  
..         ...      ...         ...       ...        ...       ...        ...  
227          0        9           0        20          0         6          0  
228          0        0           0         0          0         0          0  
229          0        0           0         0          0         0          0  
230          0        0           0         0          0         0          0  
231  57.508418      925   62.289562       828  55.757576       240  16.161616  

[232 rows x 24 columns]

## 4 años - DPT2

In [ ]:
df_4_DPT2 = preprocessing_enf_edad(df_cobertura, 292+27+27)
df_4_DPT2

,ENE,%,FEB,%,MAR,%,ABR,%,MAY,%,...,AGO,%,SEPT,%,OCT,%,NOV,%,DIC,%
1,144,3.540693,179,4.401279,103,2.532579,7,0.172117,11,0.27047,...,128,3.147283,402,9.884436,210,5.163511,477,11.728547,182,4.475043
2,54,2.302772,83,3.539446,42,1.791045,0,0,1,0.042644,...,50,2.132196,286,12.196162,117,4.989339,257,10.959488,103,4.392324
3,10,2.710027,20,5.420054,1,0.271003,0,0,0,0,...,4,1.084011,10,2.710027,4,1.084011,45,12.195122,13,3.523035
4,4,1.101928,9,2.479339,4,1.101928,0,0,1,0.275482,...,2,0.550964,15,4.132231,67,18.4573,30,8.264463,7,1.928375
5,10,2.207506,11,2.428256,9,1.986755,0,0,0,0,...,4,0.883002,121,26.710817,10,2.207506,63,13.907285,39,8.609272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,11,0,6,0,12,0,0,0,1,0,...,8,0,13,0,16,0,45,0,11,0
229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
enfermedad = 'DPT2'
rango_de_edad = '4_AÑOS'
df_4_DPT2_final = define_multilevel_col(df_4_DPT2,enfermedad, rango_de_edad)
df_4_DPT2_stacked = df_4_DPT2_final.stack(level = [0,1,2])
df_4_DPT2_final

4_AÑOS                                                           \
        DPT2                                                            
       ENERO            FEBRERO              MARZO              ABRIL   
    CANTIDAD         % CANTIDAD         % CANTIDAD         % CANTIDAD   
0        144  3.540693      179  4.401279      103  2.532579        7   
1         54  2.302772       83  3.539446       42  1.791045        0   
2         10  2.710027       20  5.420054        1  0.271003        0   
3          4  1.101928        9  2.479339        4  1.101928        0   
4         10  2.207506       11  2.428256        9  1.986755        0   
..       ...       ...      ...       ...      ...       ...      ...   
227       11         0        6         0       12         0        0   
228        0         0        0         0        0         0        0   
229        0         0        0         0        0         0        0   
230        0         0        0         0        0         0        0   
231      342  2.953368      375  3.238342      251   2.16753       36   

                                  ...                                          \
                                  ...                                           
                  MAYO            ...   AGOSTO           SETIEMBRE              
            % CANTIDAD         %  ... CANTIDAD         %  CANTIDAD          %   
0    0.172117       11   0.27047  ...      128  3.147283       402   9.884436   
1           0        1  0.042644  ...       50  2.132196       286  12.196162   
2           0        0         0  ...        4  1.084011        10   2.710027   
3           0        1  0.275482  ...        2  0.550964        15   4.132231   
4           0        0         0  ...        4  0.883002       121  26.710817   
..        ...      ...       ...  ...      ...       ...       ...        ...   
227         0        1         0  ...        8         0        13          0   
228         0        0         0  ...        0         0         0          0   
229         0        0         0  ...        0         0         0          0   
230         0        0         0  ...        0         0         0          0   
231  0.310881       63  0.544041  ...      333  2.875648       810   6.994819   

                                                                 
                                                                 
     OCTUBRE           NOVIEMBRE            DICIEMBRE            
    CANTIDAD         %  CANTIDAD          %  CANTIDAD         %  
0        210  5.163511       477  11.728547       182  4.475043  
1        117  4.989339       257  10.959488       103  4.392324  
2          4  1.084011        45  12.195122        13  3.523035  
3         67   18.4573        30   8.264463         7  1.928375  
4         10  2.207506        63  13.907285        39  8.609272  
..       ...       ...       ...        ...       ...       ...  
227       16         0        45          0        11         0  
228        0         0         0          0         0         0  
229        0         0         0          0         0         0  
230        0         0         0          0         0         0  
231      740  6.390328      1048   9.050086       396  3.419689  

[232 rows x 24 columns]

## 4 años - APO2

In [ ]:
df_4_APO2 = preprocessing_enf_edad(df_cobertura, 292+27+27+27)
df_4_APO2

,%,FEB,%,MAR,%,ABR,%,MAY,%,JUN,...,%,SEPT,%,OCT,%,NOV,%,DIC,%,TOTAL
1,2.827637,146,3.58987,90,2.212933,7,0.172117,9,0.221293,30,...,2.729284,357,8.777969,202,4.966806,378,9.29432,163,4.007868,1708
2,1.279318,54,2.302772,34,1.449893,0,0,0,0,12,...,1.876333,249,10.618337,113,4.818763,190,8.102345,86,3.667377,848
3,0.271003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,8,2.168022,0,0,9
4,0.550964,8,2.203857,4,1.101928,0,0,0,0,4,...,0.550964,11,3.030303,72,19.834711,28,7.713499,5,1.37741,146
5,1.986755,10,2.207506,8,1.766004,0,0,0,0,3,...,0.883002,113,24.944812,10,2.207506,54,11.92053,36,7.94702,254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,0,6,0,14,0,0,0,1,0,14,...,0,13,0,16,0,45,0,11,0,156
229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
enfermedad = 'APO2'
rango_de_edad = '4_AÑOS'
df_4_APO2_final = define_multilevel_col(df_4_APO2,enfermedad, rango_de_edad)
df_4_APO2_stacked = df_4_APO2_final.stack(level = [0,1,2])
df_4_APO2_final

4_AÑOS                                                                 \
         APO2                                                                  
        ENERO        FEBRERO          MARZO         ABRIL          MAYO        
     CANTIDAD    %  CANTIDAD    %  CANTIDAD   %  CANTIDAD   %  CANTIDAD    %   
0    2.827637  146   3.58987   90  2.212933   7  0.172117   9  0.221293   30   
1    1.279318   54  2.302772   34  1.449893   0         0   0         0   12   
2    0.271003    0         0    0         0   0         0   0         0    0   
3    0.550964    8  2.203857    4  1.101928   0         0   0         0    4   
4    1.986755   10  2.207506    8  1.766004   0         0   0         0    3   
..        ...  ...       ...  ...       ...  ..       ...  ..       ...  ...   
227         0    6         0   14         0   0         0   1         0   14   
228         0    0         0    0         0   0         0   0         0    0   
229         0    0         0    0         0   0         0   0         0    0   
230         0    0         0    0         0   0         0   0         0    0   
231  2.573402  335  2.892919  219  1.891192  40  0.345423  55  0.474957  102   

     ...                                                                \
     ...                                                                 
     ...    AGOSTO       SETIEMBRE         OCTUBRE      NOVIEMBRE        
     ...  CANTIDAD    %   CANTIDAD    %   CANTIDAD    %  CANTIDAD    %   
0    ...  2.729284  357   8.777969  202   4.966806  378   9.29432  163   
1    ...  1.876333  249  10.618337  113   4.818763  190  8.102345   86   
2    ...         0    0          0    0          0    8  2.168022    0   
3    ...  0.550964   11   3.030303   72  19.834711   28  7.713499    5   
4    ...  0.883002  113  24.944812   10   2.207506   54  11.92053   36   
..   ...       ...  ...        ...  ...        ...  ...       ...  ...   
227  ...         0   13          0   16          0   45         0   11   
228  ...         0    0          0    0          0    0         0    0   
229  ...         0    0          0    0          0    0         0    0   
230  ...         0    0          0    0          0    0         0    0   
231  ...  2.556131  710   6.131261  710   6.131261  892  7.702936  359   

                     
                     
    DICIEMBRE        
     CANTIDAD     %  
0    4.007868  1708  
1    3.667377   848  
2           0     9  
3     1.37741   146  
4     7.94702   254  
..        ...   ...  
227         0   156  
228         0     0  
229         0     0  
230         0     0  
231  3.100173  4289  

[232 rows x 24 columns]

## Merge

In [ ]:
df_2020_enf = pd.concat([df_rn_BCG_final, df_rn_HVB_final, df_m1_apo_final, df_m1_penta_final, df_m1_rota_final, df_m1_neumo_final, df_m1_spr_final, df_m1_dpt_final, df_m1_aporef_final, df_m1_ama_final, df_m1_vari_final, df_2_infllu_final, df_3_infllu_final, df_4_DPT2_final, df_4_APO2_final] , axis = 1)

In [ ]:
df_2020_enf.index = df_2020_enf.index +1
df_2020_enf

RN                                                              \
         BCG                                                               
       ENERO             FEBRERO               MARZO               ABRIL   
    CANTIDAD          % CANTIDAD          % CANTIDAD          % CANTIDAD   
1         55   2.442274       59   2.619893       53   2.353464      124   
2         24   1.525747       21   1.335029       17   1.080737       57   
3          2   0.505051        3   0.757576        1   0.252525        0   
4          1   0.273973        0          0        0          0        0   
5          0          0        0          0        0          0        0   
..       ...        ...      ...        ...      ...        ...      ...   
228      542          0      575          0      587          0      315   
229        0          0        0          0        0          0        0   
230        0          0        0          0        0          0        0   
231        0          0        0          0        0          0        0   
232      783  16.484211      788  16.589474      793  16.694737      628   

                                    ...    4_AÑOS                       \
                                    ...      APO2                        
                   MAYO             ...    AGOSTO       SETIEMBRE        
             % CANTIDAD          %  ...  CANTIDAD    %   CANTIDAD    %   
1     5.506217       72   3.197158  ...  2.729284  357   8.777969  202   
2     3.623649        6   0.381437  ...  1.876333  249  10.618337  113   
3            0        0          0  ...         0    0          0    0   
4            0        0          0  ...  0.550964   11   3.030303   72   
5            0        0          0  ...  0.883002  113  24.944812   10   
..         ...      ...        ...  ...       ...  ...        ...  ...   
228          0      265          0  ...         0   13          0   16   
229          0        0          0  ...         0    0          0    0   
230          0        0          0  ...         0    0          0    0   
231          0        0          0  ...         0    0          0    0   
232  13.221053      502  10.568421  ...  2.556131  710   6.131261  710   

                                                    
                                                    
       OCTUBRE      NOVIEMBRE      DICIEMBRE        
      CANTIDAD    %  CANTIDAD    %  CANTIDAD     %  
1     4.966806  378   9.29432  163  4.007868  1708  
2     4.818763  190  8.102345   86  3.667377   848  
3            0    8  2.168022    0         0     9  
4    19.834711   28  7.713499    5   1.37741   146  
5     2.207506   54  11.92053   36   7.94702   254  
..         ...  ...       ...  ...       ...   ...  
228          0   45         0   11         0   156  
229          0    0         0    0         0     0  
230          0    0         0    0         0     0  
231          0    0         0    0         0     0  
232   6.131261  892  7.702936  359  3.100173  4289  

[232 rows x 360 columns]

In [ ]:
df_final_1 = pd.concat([df_centros_de_salud,df_2020_enf],axis = 1)

In [ ]:
df_final_1.dropna(inplace = True)
df_final_1

,ESTABLECIMIENTO_DE_SALUD,AÑO,"(RN, BCG, ENERO, CANTIDAD)","(RN, BCG, ENERO, %)","(RN, BCG, FEBRERO, CANTIDAD)","(RN, BCG, FEBRERO, %)","(RN, BCG, MARZO, CANTIDAD)","(RN, BCG, MARZO, %)","(RN, BCG, ABRIL, CANTIDAD)","(RN, BCG, ABRIL, %)",...,"(4_AÑOS, APO2, AGOSTO, CANTIDAD)","(4_AÑOS, APO2, AGOSTO, %)","(4_AÑOS, APO2, SETIEMBRE, CANTIDAD)","(4_AÑOS, APO2, SETIEMBRE, %)","(4_AÑOS, APO2, OCTUBRE, CANTIDAD)","(4_AÑOS, APO2, OCTUBRE, %)","(4_AÑOS, APO2, NOVIEMBRE, CANTIDAD)","(4_AÑOS, APO2, NOVIEMBRE, %)","(4_AÑOS, APO2, DICIEMBRE, CANTIDAD)","(4_AÑOS, APO2, DICIEMBRE, %)"
3,CENTRO AMERICA,2020.0,2,0.505051,3,0.757576,1,0.252525,0,0,...,0,0,0,0,0,8,2.168022,0,0,9
4,CLAS TUPAC AMARU,2020.0,1,0.273973,0,0,0,0,0,0,...,0.550964,11,3.030303,72,19.834711,28,7.713499,5,1.37741,146
5,DOS DE MAYO,2020.0,0,0,0,0,0,0,0,0,...,0.883002,113,24.944812,10,2.207506,54,11.92053,36,7.94702,254
6,HUSARES DEL PERU,2020.0,3,1.25523,1,0.41841,0,0,1,0.41841,...,5.725191,32,12.21374,16,6.10687,14,5.343511,6,2.290076,116
7,JOSE OLAYA,2020.0,0,0,0,0,0,0,0,0,...,5.882353,41,24.117647,1,0.588235,13,7.647059,6,3.529412,89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,SAN PEDRO DE CHIO,2020.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,5.882353,5,29.411765,6
227,SINCHI ROCA,2020.0,0,0,0,0,1,0,1,0,...,0,0,0,1,4.166667,1,4.166667,0,0,14
229,HOSPITAL II PUCALLPA ESSALUD,2020.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,HOSPITAL REGIONAL DE PUCALLPA,2020.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
